In [ ]:
from importlib import reload
#reload(Utilities)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
from pathlib import Path
import json
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import copy

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
run_testing_data=True
fig_num=0

dovs_and_end_events_data_dir = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data'
if run_testing_data:
    dovs_and_end_events_data_dir = os.path.join(dovs_and_end_events_data_dir, r'TESTING_DATASETS')

files_dir_outg             = os.path.join(dovs_and_end_events_data_dir, r'EndEvents')
files_dir_outg_prim_strict = os.path.join(dovs_and_end_events_data_dir, r'EndEvents_prim_strict')
files_dir_no_outg = os.path.join(dovs_and_end_events_data_dir, r'EndEvents_NoOutg')

file_path_glob = r'end_events_[0-9]*.csv'
file_path_regex = None


assert_all_cols_equal=True
include_normalize_by_nSNs=True
inclue_zero_counts=True
return_multiindex_outg_reason=False
return_normalized_separately=False

In [ ]:
patterns_to_replace=[
    r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', 
   (
       (
           r'(Under Voltage)\s*'\
           r'([0-9a-zA-Z]*)?\s*'\
           r'(\([0-9a-zA-Z\s]*\))\s*'\
           r'([0-9a-zA-Z]*)?\s?'\
           r'(for meter\:?\s*)'\
           r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
           r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
           r'(Voltage out of tolerance)?'
       ), 
       'AMIEndEvents.under_voltage_match_func'
   ), 
   (
       (
           r'(Last Gasp\s*-\s*[0-9a-zA-Z\s]*)[\s\:,.]*'\
           r'.*'\
           r'(Fail Reason: .*)$'
       ), 
       'AMIEndEvents.last_gasp_reduce_func'
   ), 
    (r'(Angle out of tolerance) \[.*\]', r'\1'), 
    (r'(NIC Power Restore Trap Received from device).*', r'\1'), #TODO This has much info, like last gasp
    (
        (
            r'(Requested operation .* could not be applied to the given device type and firmware version.) '\
            r'Device, DeviceType: .*, Firmware Version: .*$'
        ), 
        r'\1'
    ), 
    ('meterN/A', 'meter'),
    (r'(Meter needs explicit time sync.) Drift: -?\d* s, (Encountered Problems:\s*.*), Meter_Time', r'\1 \2'), 
    (r'(Meter Program Seal mismatch for Device) \[Device ID, MAC Id\] = .*', r'\1'), 
    (r'Device Time: .* Failed Device Reason: (.*) Reboot Counter: .* Refresh Counter: .*', r'\1'), 
    (r'(Ignoring (?:Interval|Register) Read data for device as it has time in the future) .*', r'\1'), 
    (r'(Secure association operation failed consecutively for 1 times for [0-9a-zA-Z]{4}.) .*', r'\1'), 
    (r'Device, (Last Gasp State: .*), (Detector State: .*), Reboot Count: \d*', r'\1, \2'), 
    (r'(Detected end of voltage sag on meter).*', r'\1'), 
    (r'(Detected end of voltage swell on meter).*', r'\1'), 
    r'N/A', 
    (r'\s{2,}', ' ')
]

# NEW DEV

### OLD FUNCTIONS TO BE UPDATED

In [ ]:
def get_active_SNs_for_xfmrs_OLD(
    trsf_pole_nbs, 
    df_mp_curr, 
    df_mp_hist,
    no_outg_time_infos_df=None, 
    addtnl_mp_df_curr_cols=None, 
    addtnl_mp_df_hist_cols=None, 
    files_dir_no_outg=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\EndEvents_NoOutg', 
    file_path_glob_no_outg = r'end_events_[0-9]*.csv', 
    return_SNs_col='SNs', 
    return_prem_nbs_col='prem_nbs', 
    assert_all_trsf_pole_nbs_found=True, 
    df_mp_serial_number_col='mfr_devc_ser_nbr', 
    df_mp_prem_nb_col='prem_nb', 
    df_mp_install_time_col='inst_ts', 
    df_mp_removal_time_col='rmvl_ts', 
    df_mp_trsf_pole_nb_col='trsf_pole_nb', 
    t_min_col='t_min', 
    t_max_col='t_max'
):
    r"""
    Difficulty is that default.meter_premise_hist does not have trsf_pole_nb field.
    Therefore, one must use default.meter_premise to find the premise numbers for xfrms in trsf_pole_nbs,
      then use those PNs to select the correct entries from default.meter_premise_hist.
    
    If df_mp_curr OR df_mp_hist is not supplied, both will be built!
    
    addtnl_mp_df_curr_cols/addtnl_mp_df_hist_cols:
      Only used when df_mp_curr/df_mp_hist not supplied and therefore need to be built
      
    If no_outg_time_infos_df is not supplied, it will be built.
      files_dir_no_outg and file_path_glob_no_outg are only used when no_outg_time_infos_df needs built
    """
    #-------------------------
    necessary_mp_cols = [df_mp_serial_number_col, df_mp_prem_nb_col, df_mp_install_time_col, df_mp_removal_time_col]
    #-------------------------
    if df_mp_curr is None or df_mp_hist is None:
        mp_df_curr_hist = MeterPremise.build_mp_df_curr_hist_for_xfmrs(
            trsf_pole_nbs, 
            join_curr_hist=False, 
            addtnl_mp_df_curr_cols=addtnl_mp_df_curr_cols, 
            addtnl_mp_df_hist_cols=addtnl_mp_df_hist_cols, 
            df_mp_serial_number_col=df_mp_serial_number_col, 
            df_mp_prem_nb_col=df_mp_prem_nb_col, 
            df_mp_install_time_col=df_mp_install_time_col, 
            df_mp_removal_time_col=df_mp_removal_time_col, 
            df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
        )
        df_mp_curr = mp_df_curr_hist['mp_df_curr']
        df_mp_hist = mp_df_curr_hist['mp_df_hist']
    #-------------------------
    # At a bare minimum, df_mp_curr and df_mp_hist must both have the following columns:
    #   necessary_mp_cols = ['mfr_devc_ser_nbr', 'prem_nb', 'inst_ts', 'rmvl_ts']
    assert(all([x in df_mp_curr.columns for x in necessary_mp_cols+[df_mp_trsf_pole_nb_col]]))
    assert(all([x in df_mp_hist.columns for x in necessary_mp_cols]))
    #-------------------------
    # PNs_for_xfmrs is a DF with trsf_pole_nbs indices and elements which are lists of PNs for each xfmr
    PNs_for_xfmrs = MeterPremise.get_SNs_andor_PNs_for_xfmrs(
        trsf_pole_nbs=trsf_pole_nbs, 
        include_SNs=False,
        include_PNs=True,
        trsf_pole_nb_col=df_mp_trsf_pole_nb_col, 
        serial_number_col=df_mp_serial_number_col, 
        prem_nb_col=df_mp_prem_nb_col, 
        return_SNs_col=None, #Not grabbing SNs
        return_PNs_col=return_prem_nbs_col, 
        assert_all_trsf_pole_nbs_found=assert_all_trsf_pole_nbs_found, 
        mp_df=df_mp_curr, 
        return_mp_df_also=False
    )
    #-------------------------
    # Instead of a DF with trsf_pole_nb index and prem_nb column, we want opposite
    xfmr_for_PNs_df = PNs_for_xfmrs.explode(return_prem_nbs_col)
    xfmr_for_PNs_df[xfmr_for_PNs_df.index.name] = xfmr_for_PNs_df.index
    xfmr_for_PNs_df = xfmr_for_PNs_df.set_index(return_prem_nbs_col)  
    #-------------------------
    # If no_outg_time_infos_df is None, build it.  
    #   no_outg_time_infos_df has prem_nbs indices and t_min, t_max columns
    #   This is where the time information for each premise number comes from
    if no_outg_time_infos_df is None:
        paths_no_outg = Utilities.find_all_paths(base_dir=files_dir_no_outg, glob_pattern=file_path_glob_no_outg)
        no_outg_time_infos_df = MECPOAn.get_bsln_time_interval_infos_df_from_summary_files(
            summary_paths=[AMIEndEvents.find_summary_file_from_csv(x) for x in paths_no_outg], 
            output_prem_nbs_col=return_prem_nbs_col, 
            output_t_min_col=t_min_col, 
            output_t_max_col=t_max_col, 
            make_prem_nbs_idx=True, 
            include_summary_paths=False
        )    
    #-------------------------
    # Merge xfmr_for_PNs_df with no_outg_time_infos_df to append the time data to the former
    # NOTE: It is possible for t_min/t_max to be NaT (NaN) for some entries after the merge, meaning that the 
    #       premise numbers were not found in no_outg_time_infos_df
    #       This happens because these premise numbers must not have had any meter events, and thus were not included 
    #         in the SQL query (as it takes a long time to find empty results, so I weed these out before running the 
    #         query), and therefore the premise numbers were not found in the summary files/no_outg_time_infos_df.    
    xfmr_for_PNs_df = pd.merge(xfmr_for_PNs_df, no_outg_time_infos_df, how='left', left_index=True, right_index=True)
    #-------------------------
    # Want to consolidate xfmr_for_PNs_df, grouping by trsf_pole_nb and collecting t_min,t_max, and a list
    # of the premise numbers.  Therefore, first the index must be reset to make a PNs columns
    xfmr_for_PNs_df=xfmr_for_PNs_df.reset_index()
    #-----
    # Consolidate xfmr_for_PNs_df
    # NOTE: If t_min/t_max is NaT (NaN) for all premise numbers in a given trsf_pole_nb (see NOTE above before merge
    #       with no_outg_time_infos_df), then Utilities_df.consolidate_df will return an empty list (technically, an
    #       empty np.ndarray) for that trsf_pole_nb
    xfmr_for_PNs_df=Utilities_df.consolidate_df_OLD(
        df=xfmr_for_PNs_df, 
        groupby_col=df_mp_trsf_pole_nb_col, 
        cols_shared_by_group=[t_min_col, t_max_col], 
        cols_to_collect_in_lists=[return_prem_nbs_col]
    )    
    #--------------------------------------------------
    # Only reason for making dict is to ensure trsf_pole_nbs are not repeated 
    active_SNs_in_xfmrs_dfs_dict = {}

    for trsf_pole_nb_i, row_i in xfmr_for_PNs_df.iterrows():
        # active_SNs_df_i will have indices equal to premise numbers and value equal to lists
        #   of active SNs for each PN
        PNs_i=row_i[return_prem_nbs_col]
        dt_0_i=row_i[t_min_col]
        dt_1_i=row_i[t_max_col]
        # See NOTEs above regarding t_min/t_max being empty
        # In such a case, it is simply impossibe (with the summary files currently generated) to access
        #   the date over which the data would have been run, if any events existed.
        #   In future versions, this information will be included in the summary files!
        # I don't want to completely exclude these (by e.g., setting dt_0_i=pd.Timestamp.min and 
        #   dt_1_i=pd.Timestamp.max), so I will simply include the meters which are active TODAY.
        # This obviously is not correct, but this occurrence is rare (only happening when every single meter
        #   on a transformer had no events during the time period) and this crude approximation will be fine.
        if Utilities.is_object_one_of_types(dt_0_i, [list, np.ndarray]):
            assert(len(dt_0_i)==0)
            # I believe if this happens for one it should happen for both...
            assert(Utilities.is_object_one_of_types(dt_1_i, [list, np.ndarray]) and len(dt_1_i)==0)
            dt_0_i=pd.Timestamp.today()
        if Utilities.is_object_one_of_types(dt_1_i, [list, np.ndarray]):
            assert(len(dt_1_i)==0)
            # I believe if this happens for one it should happen for both...
            # But, dt_0_i changed already above, so much check row_i[t_min_col] instead!
            assert(Utilities.is_object_one_of_types(row_i[t_min_col], [list, np.ndarray]) and len(row_i[t_min_col])==0)
            dt_1_i=pd.Timestamp.today()
        active_SNs_df_i = MeterPremise.get_active_SNs_for_PNs_at_datetime_interval(
            PNs=PNs_i,
            df_mp_curr=df_mp_curr, 
            df_mp_hist=df_mp_hist, 
            dt_0=dt_0_i,
            dt_1=dt_1_i,
            output_index=None,
            output_groupby=[df_mp_prem_nb_col], 
            include_prems_wo_active_SNs_when_groupby=True, 
            assert_all_PNs_found=False
        )
        active_SNs_df_i=active_SNs_df_i.reset_index()
        if active_SNs_df_i.shape[0]==0:
            active_SNs_df_i[df_mp_prem_nb_col] = np.nan
            active_SNs_df_i[df_mp_serial_number_col] = [[]] 
        active_SNs_df_i[df_mp_trsf_pole_nb_col] = trsf_pole_nb_i
        active_SNs_df_i = active_SNs_df_i.explode(df_mp_serial_number_col)
        assert(trsf_pole_nb_i not in active_SNs_in_xfmrs_dfs_dict)
        active_SNs_in_xfmrs_dfs_dict[trsf_pole_nb_i] = active_SNs_df_i
    #-------------------------
    active_SNs_df = pd.concat(list(active_SNs_in_xfmrs_dfs_dict.values()))
    #-------------------------
    active_SNs_df = Utilities_df.consolidate_df_OLD(
        df=active_SNs_df, 
        groupby_col=df_mp_trsf_pole_nb_col, 
        cols_shared_by_group=None, 
        cols_to_collect_in_lists=[df_mp_serial_number_col, df_mp_prem_nb_col], 
        include_groupby_col_in_output_cols=False, 
        allow_duplicates_in_lists=False, 
        recover_uniqueness_violators=True, 
        rename_cols=None, 
        verbose=False
    )
    #-----
    # Change [nan] entries to []
    active_SNs_df.loc[active_SNs_df[df_mp_serial_number_col].apply(lambda x: len([ix for ix in x if not pd.isna(ix)]))==0, df_mp_serial_number_col] = [[]]
    active_SNs_df.loc[active_SNs_df[df_mp_prem_nb_col].apply(lambda x: len([ix for ix in x if not pd.isna(ix)]))==0, df_mp_prem_nb_col] = [[]]
    #-------------------------
    active_SNs_df = active_SNs_df.rename(columns={
        df_mp_prem_nb_col:return_prem_nbs_col, 
        df_mp_serial_number_col:return_SNs_col
    })
    #-------------------------
    return active_SNs_df

In [ ]:
def add_xfmr_active_SNs_to_rcpo_df_OLD(
    rcpo_df, 
    set_xfmr_nSNs=True, 
    include_active_xfmr_PNs=False, #Should be equal to the PNs already in rcpo_df!
    df_mp_curr=None,
    df_mp_hist=None, 
    no_outg_time_infos_df=None, 
    addtnl_get_active_SNs_for_xfmrs_kwargs=None, 
    xfmr_SNs_col='_xfmr_SNs', 
    xfmr_nSNs_col='_xfmr_nSNs', 
    xfmr_PNs_col='_xfmr_PNs', 
    xfmr_nPNs_col='_xfmr_nPNs', 
):
    r"""
    NOTE: If include_active_xfmr_PNs is True, this column (named xfmr_PNs_col='_xfmr_SNs') should be 
          equal to the PNs already in rcpo_df!
    NOTE: xfmr_SNs_col, xfmr_nSNs_col, xfmr_PNs_col, and xfmr_nPNs_col should all be strings, not tuples.
          If column is multiindex, the level_0 value will be handled below.
          
    NOTE: If any of xfmr_SNs_col, xfmr_nSNs_col, xfmr_PNs_col, and xfmr_nPNs_col are already contained in 
          rcpo_df, they will be replaced.  This is needed so that the merge operation does not come back with _x and _y
          values.  So, one should make sure this function call is truly needed, as grabbing the serial numbers for the
          outages typically takes a couple/few minutes.
          
    NOTE: To make things run faster, the user can supply df_mp_curr and df_mp_hist.  These will be included in 
          get_active_SNs_for_xfmrs_kwargs.
          NOTE: If df_mp_curr/df_mp_hist is also supplied in addtnl_get_active_SNs_for_xfmrs_kwargs,
                that/those in addtnl_get_active_SNs_for_xfmrs_kwargs will ultimately be used (not the
                explicity df_mp_hist/curr in the function arguments!)
          CAREFUL: If one does supple df_mp_curr/hist, one must be certain these DFs contain all necessary elements!
    """
    #-------------------------
    get_active_SNs_for_xfmrs_kwargs = dict(
        trsf_pole_nbs=rcpo_df.index.unique().tolist(), 
        df_mp_curr=df_mp_curr, 
        df_mp_hist=df_mp_hist, 
        no_outg_time_infos_df=no_outg_time_infos_df, 
        return_prem_nbs_col=xfmr_PNs_col, 
        return_SNs_col=xfmr_SNs_col
    )
    if addtnl_get_active_SNs_for_xfmrs_kwargs is not None:
        get_active_SNs_for_xfmrs_kwargs = {**get_active_SNs_for_xfmrs_kwargs, 
                                           **addtnl_get_active_SNs_for_xfmrs_kwargs}
    active_SNs_df = get_active_SNs_for_xfmrs_OLD(**get_active_SNs_for_xfmrs_kwargs)
    assert(isinstance(active_SNs_df, pd.DataFrame))
    #-------------------------
    # Assert below might be too strong here...
    assert(sorted(rcpo_df.index.unique().tolist())==sorted(active_SNs_df.index.unique().tolist()))
    assert(rcpo_df.columns.nlevels<=2)
    if rcpo_df.columns.nlevels==1:
        #----------
        # See note above about columns being replaced/dropped
        cols_to_drop = [x for x in rcpo_df.columns if x in active_SNs_df.columns]
        if len(cols_to_drop)>0:
            rcpo_df = rcpo_df.drop(columns=cols_to_drop)
        #----------
        rcpo_df = rcpo_df.merge(active_SNs_df, left_index=True, right_index=True)
        #----------
        if set_xfmr_nSNs:
            rcpo_df = MECPODf.set_nSNs_from_SNs_in_rcpo_df(rcpo_df, xfmr_SNs_col, xfmr_nSNs_col)
            if include_active_xfmr_PNs:
                rcpo_df = MECPODf.set_nSNs_from_SNs_in_rcpo_df(rcpo_df, xfmr_PNs_col, xfmr_nPNs_col)
    else:
        # Currently, only expecting raw and/or norm.  No problem to allow more, but for now keep this to alert 
        # of anything unexpected
        assert(rcpo_df.columns.get_level_values(0).nunique()<=2)
        for i,level_0_val in enumerate(rcpo_df.columns.get_level_values(0).unique()):
            if i==0:
                active_SNs_df.columns = pd.MultiIndex.from_product([[level_0_val], active_SNs_df.columns])
            else:
                active_SNs_df.columns = active_SNs_df.columns.set_levels([level_0_val], level=0)
            #----------
            # See note above about columns being replaced/dropped
            cols_to_drop = [x for x in rcpo_df.columns if x in active_SNs_df.columns]
            if len(cols_to_drop)>0:
                rcpo_df = rcpo_df.drop(columns=cols_to_drop)
            #----------
            rcpo_df = rcpo_df.merge(active_SNs_df, left_index=True, right_index=True)
            #----------
            if set_xfmr_nSNs:
                rcpo_df = MECPODf.set_nSNs_from_SNs_in_rcpo_df(rcpo_df, (level_0_val, xfmr_SNs_col), (level_0_val, xfmr_nSNs_col))
                if include_active_xfmr_PNs:
                    rcpo_df = MECPODf.set_nSNs_from_SNs_in_rcpo_df(rcpo_df, (level_0_val, xfmr_PNs_col), (level_0_val, xfmr_nPNs_col))
    #-------------------------
    rcpo_df = rcpo_df.sort_index(axis=1,level=0)
    #-------------------------
    return rcpo_df

In [ ]:
def build_rcpo_df_norm_by_xfmr_active_nSNs_OLD(
    rcpo_df_raw, 
    xfmr_nSNs_col='_xfmr_nSNs', 
    xfmr_SNs_col='_xfmr_SNs', 
    other_SNs_col_tags_to_ignore=['_SNs', '_nSNs', '_prem_nbs', '_nprem_nbs', '_xfmr_PNs', '_xfmr_nPNs'], 
    drop_xfmr_nSNs_eq_0=True, 
    new_level_0_val='counts_norm_by_xfmr_nSNs', 
    remove_SNs_cols=False, 
    df_mp_curr=None,
    df_mp_hist=None, 
    no_outg_time_infos_df=None, 
    addtnl_get_active_SNs_for_xfmrs_kwargs=None
):
    r"""
    Build rcpo_df normalized by the number of serial numbers in each outage

    drop_xfmr_nSNs_eq_0:
      It is possible for the number of serial numbers in an outage to be zero!
      Premise numbers are always found, but the meter_premise database does not always 
        contain the premise numbers.
      Dividing by zero will make all counts for such an entry equal to NaN or inf.
      When drop_xfmr_nSNs_eq_0 is True, such entries will be removed.

    NOTE: xfmr_SNs_col and xfmr_nSNs_col should both be strings, not tuples.
          If column is MultiIndex, the level_0 value will be handled below.
    """
    #-------------------------
    n_counts_col = xfmr_nSNs_col
    list_col = xfmr_SNs_col
    #-------------------------
    # NOTE: MECPODf.add_outage_SNs_to_rcpo_df expects xfmr_SNs_col and xfmr_nSNs_col to be strings, not tuples
    #       as it handles the level 0 values if they exist.  So, if tuples, use only highest level values (i.e., level 1)
    assert(Utilities.is_object_one_of_types(list_col, [str, tuple]))
    assert(Utilities.is_object_one_of_types(n_counts_col, [str, tuple]))
    #-----
    add_list_col_to_rcpo_df_func = add_xfmr_active_SNs_to_rcpo_df_OLD
    add_list_col_to_rcpo_df_kwargs = dict(
        set_xfmr_nSNs=True, 
        include_active_xfmr_PNs=True, 
        df_mp_curr=df_mp_curr,
        df_mp_hist=df_mp_hist, 
        no_outg_time_infos_df=no_outg_time_infos_df, 
        addtnl_get_active_SNs_for_xfmrs_kwargs=addtnl_get_active_SNs_for_xfmrs_kwargs, 
        xfmr_SNs_col='_xfmr_SNs', 
        xfmr_nSNs_col='_xfmr_nSNs', 
        xfmr_PNs_col='_xfmr_PNs', 
        xfmr_nPNs_col='_xfmr_nPNs', 
    )
    #-------------------------
    other_col_tags_to_ignore = other_SNs_col_tags_to_ignore
    drop_n_counts_eq_0 = drop_xfmr_nSNs_eq_0
    new_level_0_val = new_level_0_val
    remove_ignored_cols = remove_SNs_cols
    #-------------------------
    return MECPODf.build_rcpo_df_norm_by_list_counts(
        rcpo_df_raw=rcpo_df_raw, 
        n_counts_col=n_counts_col, 
        list_col=list_col, 
        add_list_col_to_rcpo_df_func=add_list_col_to_rcpo_df_func, 
        add_list_col_to_rcpo_df_kwargs=add_list_col_to_rcpo_df_kwargs, 
        other_col_tags_to_ignore=other_col_tags_to_ignore, 
        drop_n_counts_eq_0=drop_n_counts_eq_0, 
        new_level_0_val=new_level_0_val, 
        remove_ignored_cols=remove_ignored_cols
    )

### NEW FUNCTIONS

In [ ]:
def reset_index_and_identify_cols_to_merge(
    df, 
    merge_on, 
    tag_for_idx_names=None
):
    r"""
    Designed to work with merge_rcpo_and_df (but should definitely be useful elsewhere), which 
    allows the user to join the DFs by columns, specific index levels or any mixture of the two.
    
    In order to achieve this type of general merging, it is easiest to call reset_index, making everything
      to be merged on a column.
    HOWEVER, in order to keep track of the original indices, which likely will be restored after the merge,
      it is important for all index levels to have a name.
      
    merge_on:
        This is a list that directs the columns/indices to be used in the join.
        Column identifiers:
            Single strings for normal DF, lists/tuples for MultiIndex columns
        Index identifiers:
            f'index_{idx_level}' to specify index level by number
            ('index', idx_level_name) to specify index level by name
            
    RETURNS:
        A dict object with keys = ['df', 'df_idx_names_OG', 'df_idx_names', 'reset_merge_on']
    """
    #-------------------------
    df=df.copy()
    #-------------------------
    assert(Utilities.is_object_one_of_types(merge_on, [list, tuple]))
    #-------------------------
    # Make sure all index levels have names!
    # If the level has a name, the code below will leave it unchanged in df_idx_names
    # If it does not have a name, it will be names f'index_{idx_level}', where idx_level 
    #   is the index level number
    df_idx_names_OG = list(df.index.names)
    df_idx_names = [x if x is not None else f'index_{i}' 
                    for i,x in enumerate(df_idx_names_OG)]
    if tag_for_idx_names is not None:
        df_idx_names = [f'{x}_{tag_for_idx_names}' for x in df_idx_names]
    df.index.names=df_idx_names
    #-------------------------
    reset_merge_on = []
    for idfr in merge_on:
        assert(Utilities.is_object_one_of_types(idfr, [str, list, tuple]))
        if idfr in df.columns:
            reset_merge_on.append(idfr)
        else:
            # Must be in indices!
            if isinstance(idfr, str):
                assert(idfr.startswith('index'))
                if idfr=='index':
                    idfr_idx_lvl=0
                else:
                    idfr_idx_lvl = re.findall('index_(\d*)', idfr)
                    assert(len(idfr_idx_lvl)==1)
                    idfr_idx_lvl=idfr_idx_lvl[0]
                    idfr_idx_lvl=int(idfr_idx_lvl)
            else:
                assert(len(idfr)==2)
                assert(idfr[0]=='index')
                idx_level_name = idfr[1]
                # If tag_for_idx_names, df.index.names already changed, so idx_level_name must be adjusted
                if tag_for_idx_names is not None:
                    idx_level_name = f'{idx_level_name}_{tag_for_idx_names}'
                assert(idx_level_name in df.index.names)
                idfr_idx_lvl = df.index.names.index(idx_level_name)
            #---------------
            assert(idfr_idx_lvl < df.index.nlevels)
            reset_merge_on_i = df.index.names[idfr_idx_lvl]
            # NOTE: If df.columns.nlevels>1, then calling df.reset_index() below
            #       will make the bottom level reset_merge_on_i and all the rest ''
            #       e.g., if nlevels=2, after df.reset_index(), reset_merge_on_i--> (reset_merge_on_i, '')
            if df.columns.nlevels>1:
                reset_merge_on_i=tuple([reset_merge_on_i] + ['']*(df.columns.nlevels-1))
            reset_merge_on.append(reset_merge_on_i)
    #-------------------------
    # Call reset_index on df, making all indices into columns, and double check that all reset_merge_on
    #   are contained in the columns
    df = df.reset_index()
    assert(len(set(reset_merge_on).difference(set(df.columns)))==0)
    #-------------------------
    return dict(
        df=df, 
        df_idx_names_OG=df_idx_names_OG, 
        df_idx_names=df_idx_names, 
        reset_merge_on=reset_merge_on
    )


def merge_rcpo_and_df(
    rcpo_df, 
    df_2, 
    rcpo_df_on,
    df_2_on, 
    how='left'
):
    r"""
    Merge together rcpo_df and df_2 dfs.
    Designed specifically for rcpo_df and time_infos_df.
    
    rcpo_df_on/df_2_on:
        These are lists which direct the columns/indices to be used in the join.
        Column identifiers:
            Single strings for normal DF, lists/tuples for MultiIndex columns
        Index identifiers:
            f'index_{idx_level}' to specify index level by number
            ('index', idx_level_name) to specify index level by name
            
        NOTE: Calling reset_index() on both DFs will be the easiest method for this type of general merging, in
              which the DFs can be merged by columns, specific index levels or any mixture of the two.
              This is done through the use of reset_index_and_identify_cols_to_merge
    """
    #--------------------------------------------------
    # Only expecting at most 2 levels in columns (e.g., counts or counts_norm as level 0, and reason as level 1)
    #     - probably not a necessary assertion, if function to be expanded later
    #   Ultimately, the number of levels in df_2 will match that of rcpo_df, so making the same
    #     assertion on df_2
    assert((rcpo_df.columns.nlevels <= 2) and (df_2.columns.nlevels <= 2))
    if rcpo_df.columns.nlevels==2:
        # If rcpo_df has two column levels, df_2 must also for the proper merging to occur
        #   --Merging with unequal levels will cause all levels to be collapsed down to single dimension
        #-----
        # Expecting at most 2 unique values for level 0 (definitely not a necessary assertion)
        assert(rcpo_df.columns.get_level_values(0).nunique()<=2)
        # In this case, likely df_2 has only single level columns.
        #   For proper merge, the number of levels should match
        if df_2.columns.nlevels==1:
            level_0_vals = rcpo_df.columns.get_level_values(0).unique().tolist()
            # Add new top level to df_2 with value equal to level_0_vals[0]
            df_2=Utilities_df.prepend_level_to_MultiIndex(
                df=df_2, level_val=level_0_vals[0], level_name=None, axis=1
            )
            if len(level_0_vals)>1:
                # Grab df without new column level to be copied to other new column level values
                # NOTE: If extra [] placed around level_0_vals[0] below, the new column level would be returned
                #       (which is not desired here!)
                df_0 = df_2[level_0_vals[0]]
                df_0_cols = df_0.columns.tolist()
                # Reproduce the entries of df_2 for all column level 0 values in rcpo_df
                for i_lvl in range(1,len(level_0_vals)):
                    new_cols = pd.MultiIndex.from_product([[level_0_vals[i_lvl]], df_0_cols])
                    df_2[new_cols] = df_0.copy()
        # Now, at this stage, rcpo_df and df_2 should have the same number of column levels
        #   and should have overlapping level 0 values if nlevels>1
        assert((rcpo_df.columns.nlevels <= 2) and (df_2.columns.nlevels <= 2)) #not needed
        assert(rcpo_df.columns.nlevels == df_2.columns.nlevels)
        if rcpo_df.columns.nlevels == 2:
            # Make sure overlapping 0 values.  
            # I suppose user could supply df_2 with only a single level 0 value while rcpo_df has
            #   two values.  In this case, only the single value would be merged to rcpo_df
            assert(len(set(df_2.columns.get_level_values(0).unique().tolist()).difference(
                set(rcpo_df.columns.get_level_values(0).unique().tolist())
            ))==0)
    else:
        assert(rcpo_df.columns.nlevels==df_2.columns.nlevels==1)
    #--------------------------------------------------
    reset_rcpo_df_dict = reset_index_and_identify_cols_to_merge(
        df=rcpo_df, 
        merge_on=rcpo_df_on, 
        tag_for_idx_names='from_rcpo_df'
    )

    reset_df_2_dict = reset_index_and_identify_cols_to_merge(
        df=df_2, 
        merge_on=df_2_on, 
        tag_for_idx_names='from_df_2'
    )
    #-------------------------
    merged_df = pd.merge(
        reset_rcpo_df_dict['df'], 
        reset_df_2_dict['df'], 
        left_on=reset_rcpo_df_dict['reset_merge_on'], 
        right_on = reset_df_2_dict['reset_merge_on'], 
        how=how
    )
    #-------------------------
    # Want to set index first before changing names back to originals, as the
    #   originals could have been None
    merged_df = merged_df.set_index(reset_rcpo_df_dict['df_idx_names'])

    # Two lines below (instead of calling simply df.index.names=df_idx_names_OG) 
    #   ensures the order is the same
    rcpo_df_rename_dict = dict(zip(reset_rcpo_df_dict['df_idx_names'], reset_rcpo_df_dict['df_idx_names_OG']))
    merged_df.index.names = [rcpo_df_rename_dict[x] for x in merged_df.index.names]
    #-------------------------
    # When merging two columns whose names are different, both columns are kept
    #   This is redundant, as these will have identical values, as they were merged, so get rid of
    #   Note: If the column names are the same, this is obviously not an issue (hence the need to find
    #         cols_to_drop below, instead of simply dropping all of reset_df_2_dict['reset_merge_on']
    cols_to_drop = [x for x in reset_df_2_dict['reset_merge_on'] if x in merged_df.columns]
    merged_df = merged_df.drop(columns=cols_to_drop)

    # Rename columns from df_2 to original values (if original values were not None!)
    df_2_rename_dict = dict(zip(reset_df_2_dict['df_idx_names'], reset_df_2_dict['df_idx_names_OG']))
    df_2_rename_dict = {k:v for k,v in df_2_rename_dict.items() if k in merged_df.columns and v is not None}
    merged_df=merged_df.rename(columns=df_2_rename_dict)
    #-------------------------
    return merged_df

In [ ]:
# TODO still needs work...
# This replaces get_active_SNs_for_xfmrs_OLD (but, typically use get_active_SNs_for_xfmrs_in_rcpo_df)
def get_active_SNs_for_xfmrs(
    trsf_pole_nbs,     
    df_mp_curr, 
    df_mp_hist,
    time_infos_df,     
    time_infos_to_PNs = ['index'], 
    PNs_to_time_infos = ['index'], 
    how='left',     
    output_trsf_pole_nb_col=None, 
    addtnl_mp_df_curr_cols=None, 
    addtnl_mp_df_hist_cols=None, 
    return_SNs_col='SNs', 
    return_prem_nbs_col='prem_nbs', 
    assert_all_trsf_pole_nbs_found=True, 
    df_mp_serial_number_col='mfr_devc_ser_nbr', 
    df_mp_prem_nb_col='prem_nb', 
    df_mp_install_time_col='inst_ts', 
    df_mp_removal_time_col='rmvl_ts', 
    df_mp_trsf_pole_nb_col='trsf_pole_nb', 
    t_min_col='t_min', 
    t_max_col='t_max'
):
    r"""
    Difficulty is that default.meter_premise_hist does not have trsf_pole_nb field.
    Therefore, one must use default.meter_premise to find the premise numbers for xfrms in trsf_pole_nbs,
      then use those PNs to select the correct entries from default.meter_premise_hist.
      
    time_infos_to_PNs:
      Defines how time_infos_df and PNs_for_xfmrs will be merged.
      NOTE: PNs_for_xfmrs will have indices equal to trsf_pole_nbs and values equal to lists of associated prem_nbs
      See merge_rcpo_and_df and reset_index_and_identify_cols_to_merge for more information

    If df_mp_curr OR df_mp_hist is not supplied, both will be built!
    
    addtnl_mp_df_curr_cols/addtnl_mp_df_hist_cols:
      Only used when df_mp_curr/df_mp_hist not supplied and therefore need to be built
      
    """
    #--------------------------------------------------
    assert(t_min_col in time_infos_df.columns and 
           t_max_col in time_infos_df.columns)
    time_infos_df = time_infos_df[[t_min_col, t_max_col]]
    #--------------------------------------------------
    #-------------------------
    necessary_mp_cols = [df_mp_serial_number_col, df_mp_prem_nb_col, df_mp_install_time_col, df_mp_removal_time_col]
    #-------------------------
    if df_mp_curr is None or df_mp_hist is None:
        mp_df_curr_hist = MeterPremise.build_mp_df_curr_hist_for_xfmrs(
            trsf_pole_nbs, 
            join_curr_hist=False, 
            addtnl_mp_df_curr_cols=addtnl_mp_df_curr_cols, 
            addtnl_mp_df_hist_cols=addtnl_mp_df_hist_cols, 
            df_mp_serial_number_col=df_mp_serial_number_col, 
            df_mp_prem_nb_col=df_mp_prem_nb_col, 
            df_mp_install_time_col=df_mp_install_time_col, 
            df_mp_removal_time_col=df_mp_removal_time_col, 
            df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
        )
        df_mp_curr = mp_df_curr_hist['mp_df_curr']
        df_mp_hist = mp_df_curr_hist['mp_df_hist']
    #-------------------------
    # At a bare minimum, df_mp_curr and df_mp_hist must both have the following columns:
    #   necessary_mp_cols = ['mfr_devc_ser_nbr', 'prem_nb', 'inst_ts', 'rmvl_ts']
    assert(all([x in df_mp_curr.columns for x in necessary_mp_cols+[df_mp_trsf_pole_nb_col]]))
    assert(all([x in df_mp_hist.columns for x in necessary_mp_cols]))
    #-------------------------
    # PNs_for_xfmrs is a DF with trsf_pole_nbs indices and elements which are lists of PNs for each xfmr
    PNs_for_xfmrs = MeterPremise.get_SNs_andor_PNs_for_xfmrs(
        trsf_pole_nbs=trsf_pole_nbs, 
        include_SNs=False,
        include_PNs=True,
        trsf_pole_nb_col=df_mp_trsf_pole_nb_col, 
        serial_number_col=df_mp_serial_number_col, 
        prem_nb_col=df_mp_prem_nb_col, 
        return_SNs_col=None, #Not grabbing SNs
        return_PNs_col=return_prem_nbs_col, 
        assert_all_trsf_pole_nbs_found=assert_all_trsf_pole_nbs_found, 
        mp_df=df_mp_curr, 
        return_mp_df_also=False
    )
    #-------------------------
    # Join together time_infos_df and PNs_for_xfmrs
    #-----
    time_infos_df = merge_rcpo_and_df(
        rcpo_df=time_infos_df, 
        df_2=PNs_for_xfmrs, 
        rcpo_df_on=time_infos_to_PNs,
        df_2_on=PNs_to_time_infos, 
        how=how
    )
    #--------------------------------------------------
    # Only reason for making dict is to ensure trsf_pole_nbs are not repeated 
    active_SNs_in_xfmrs_dfs_dict = {}
    if output_trsf_pole_nb_col is None:
        output_trsf_pole_nb_col='trsf_pole_nb'
    for trsf_pole_nb in trsf_pole_nbs:
        # active_SNs_df_i will have indices equal to premise numbers and value equal to lists
        #   of active SNs for each PN
        PNs_i=time_infos_df.loc[trsf_pole_nb, return_prem_nbs_col]
        dt_0_i=time_infos_df.loc[trsf_pole_nb, t_min_col]
        dt_1_i=time_infos_df.loc[trsf_pole_nb, t_max_col]
        #-----
        # See NOTEs above regarding t_min/t_max being empty
        # In such a case, it is simply impossibe (with the summary files currently generated) to access
        #   the date over which the data would have been run, if any events existed.
        #   In future versions, this information will be included in the summary files!
        # I don't want to completely exclude these (by e.g., setting dt_0_i=pd.Timestamp.min and 
        #   dt_1_i=pd.Timestamp.max), so I will simply include the meters which are active TODAY.
        # This obviously is not correct, but this occurrence is rare (only happening when every single meter
        #   on a transformer had no events during the time period) and this crude approximation will be fine.
        if Utilities.is_object_one_of_types(dt_0_i, [list, np.ndarray]):
            assert(len(dt_0_i)==0)
            # I believe if this happens for one it should happen for both...
            assert(Utilities.is_object_one_of_types(dt_1_i, [list, np.ndarray]) and len(dt_1_i)==0)
            dt_0_i=pd.Timestamp.today()
        if Utilities.is_object_one_of_types(dt_1_i, [list, np.ndarray]):
            assert(len(dt_1_i)==0)
            # I believe if this happens for one it should happen for both...
            # But, dt_0_i changed already above, so must check time_infos_df.loc[trsf_pole_nb, t_min_col] instead!
            assert(Utilities.is_object_one_of_types(time_infos_df.loc[trsf_pole_nb, t_min_col], [list, np.ndarray]) and 
                   len(time_infos_df.loc[trsf_pole_nb, t_min_col])==0)
            dt_1_i=pd.Timestamp.today()
        active_SNs_df_i = MeterPremise.get_active_SNs_for_PNs_at_datetime_interval(
            PNs=PNs_i,
            df_mp_curr=df_mp_curr, 
            df_mp_hist=df_mp_hist, 
            dt_0=dt_0_i,
            dt_1=dt_1_i,
            output_index=None,
            output_groupby=[df_mp_prem_nb_col], 
            include_prems_wo_active_SNs_when_groupby=True, 
            assert_all_PNs_found=False
        )
        active_SNs_df_i=active_SNs_df_i.reset_index()
        if active_SNs_df_i.shape[0]==0:
            active_SNs_df_i[df_mp_prem_nb_col] = np.nan
            active_SNs_df_i[df_mp_serial_number_col] = [[]] 
            active_SNs_df_i[output_trsf_pole_nb_col] = trsf_pole_nb
            active_SNs_df_i = active_SNs_df_i.set_index(output_trsf_pole_nb_col)
        else:
            active_SNs_df_i[output_trsf_pole_nb_col] = trsf_pole_nb
            active_SNs_df_i = active_SNs_df_i.explode(df_mp_serial_number_col)
            active_SNs_df_i = Utilities_df.consolidate_df(
                df=active_SNs_df_i, 
                groupby_cols=[output_trsf_pole_nb_col], 
                cols_shared_by_group=None, 
                cols_to_collect_in_lists=[df_mp_serial_number_col, df_mp_prem_nb_col], 
                include_groupby_cols_in_output_cols=False, 
                allow_duplicates_in_lists=False, 
                recover_uniqueness_violators=True, 
                rename_cols=None, 
                verbose=False
            )
        assert(trsf_pole_nb not in active_SNs_in_xfmrs_dfs_dict)
        active_SNs_in_xfmrs_dfs_dict[trsf_pole_nb] = active_SNs_df_i
    #-------------------------
    active_SNs_df = pd.concat(list(active_SNs_in_xfmrs_dfs_dict.values()))
    #-------------------------
    # Change [nan] entries to []
    found_nans_srs = active_SNs_df[df_mp_serial_number_col].apply(lambda x: len([ix for ix in x if not pd.isna(ix)]))==0
    if found_nans_srs.sum()>0:
        active_SNs_df.loc[found_nans_srs, df_mp_serial_number_col] = [[]]
    #-----
    found_nans_srs = active_SNs_df[df_mp_prem_nb_col].apply(lambda x: len([ix for ix in x if not pd.isna(ix)]))==0
    if found_nans_srs.sum()>0:
        active_SNs_df.loc[found_nans_srs, df_mp_prem_nb_col] = [[]]
    #-------------------------
    active_SNs_df = active_SNs_df.rename(columns={
        df_mp_prem_nb_col:return_prem_nbs_col, 
        df_mp_serial_number_col:return_SNs_col
    })
    #-------------------------
    return active_SNs_df

In [ ]:
# Replaces get_active_SNs_for_xfmrs_OLD, but should probably build get_active_SNs_for_xfmrs
#  which accepts a list of trsf_pole_nbs instead of rcpo_df, which this function can use
def get_active_SNs_for_xfmrs_in_rcpo_df(
    rcpo_df, 
    trsf_pole_nbs_loc, 
    df_mp_curr, 
    df_mp_hist,
    time_infos_df, 
    rcpo_df_to_time_infos_on = [('index', 'outg_rec_nb')], 
    time_infos_to_rcpo_df_on = ['index'], 
    how='left', 
    rcpo_df_to_PNs_on = [('index', 'trsf_pole_nb')], 
    PNs_to_rcpo_df_on = ['index'], 
    addtnl_mp_df_curr_cols=None, 
    addtnl_mp_df_hist_cols=None, 
    return_SNs_col='SNs', 
    return_prem_nbs_col='prem_nbs', 
    assert_all_trsf_pole_nbs_found=True, 
    df_mp_serial_number_col='mfr_devc_ser_nbr', 
    df_mp_prem_nb_col='prem_nb', 
    df_mp_install_time_col='inst_ts', 
    df_mp_removal_time_col='rmvl_ts', 
    df_mp_trsf_pole_nb_col='trsf_pole_nb', 
    t_min_col='t_min', 
    t_max_col='t_max'
):
    r"""
    Difficulty is that default.meter_premise_hist does not have trsf_pole_nb field.
    Therefore, one must use default.meter_premise to find the premise numbers for xfrms in trsf_pole_nbs,
      then use those PNs to select the correct entries from default.meter_premise_hist.
    The trsf_pole_nbs should be contained in rcpo_df, and will be found using the trsf_pole_nbs_loc
      parameter described below.
      
    trsf_pole_nbs_loc:
        Directs where the transformer pole numbers are located
        This should identify an index (w/ level)
        Set equal to 'index' for normal DFs, or when trsf_pole_nbs are in level 0 of index.
        For a DF with MultiIndex index, there are two options:
            i.  Set equal to f'index_{idx_level}' for a DF with MutliIndex index, where idx_level
                is an int identifying the level in which the trsf_pole_nbs reside
            ii. Set equal to the tuple ('index', trsf_pole_nbs_idx_name), where trsf_pole_nbs_idx_name is
            the name of the index level in which the trsf_pole_nbs reside.

    If df_mp_curr OR df_mp_hist is not supplied, both will be built!
    
    addtnl_mp_df_curr_cols/addtnl_mp_df_hist_cols:
      Only used when df_mp_curr/df_mp_hist not supplied and therefore need to be built
      
    """
    #--------------------------------------------------
    assert(t_min_col in time_infos_df.columns and 
           t_max_col in time_infos_df.columns)
    time_infos_df = time_infos_df[[t_min_col, t_max_col]]
    #--------------------------------------------------
    # trsf_pole_nbs_loc can be a string or tuple/list
    # First, find trsf_pole_nbs and trsf_pole_nbs_idx_lvl
    assert(Utilities.is_object_one_of_types(trsf_pole_nbs_loc, [str, list, tuple]))
    if isinstance(trsf_pole_nbs_loc, str):
        assert(trsf_pole_nbs_loc.startswith('index'))
        if trsf_pole_nbs_loc=='index':
            trsf_pole_nbs_idx_lvl = 0
        else:
            trsf_pole_nbs_idx_lvl = re.findall('index_(\d*)', trsf_pole_nbs_loc)
            assert(len(trsf_pole_nbs_idx_lvl)==1)
            trsf_pole_nbs_idx_lvl=trsf_pole_nbs_idx_lvl[0]
            trsf_pole_nbs_idx_lvl=int(trsf_pole_nbs_idx_lvl)
    else:
        assert(len(trsf_pole_nbs_loc)==2)
        assert(trsf_pole_nbs_loc[0]=='index')
        assert(trsf_pole_nbs_loc[1] in rcpo_df.index.names)
        trsf_pole_nbs_idx_lvl = rcpo_df.index.names.index(trsf_pole_nbs_loc[1])
        #---------------
        assert(trsf_pole_nbs_idx_lvl < rcpo_df.index.nlevels)
        trsf_pole_nbs = rcpo_df.index.get_level_values(trsf_pole_nbs_idx_lvl).tolist()
    #--------------------------------------------------
    #-------------------------
    necessary_mp_cols = [df_mp_serial_number_col, df_mp_prem_nb_col, df_mp_install_time_col, df_mp_removal_time_col]
    #-------------------------
    if df_mp_curr is None or df_mp_hist is None:
        mp_df_curr_hist = MeterPremise.build_mp_df_curr_hist_for_xfmrs(
            trsf_pole_nbs, 
            join_curr_hist=False, 
            addtnl_mp_df_curr_cols=addtnl_mp_df_curr_cols, 
            addtnl_mp_df_hist_cols=addtnl_mp_df_hist_cols, 
            df_mp_serial_number_col=df_mp_serial_number_col, 
            df_mp_prem_nb_col=df_mp_prem_nb_col, 
            df_mp_install_time_col=df_mp_install_time_col, 
            df_mp_removal_time_col=df_mp_removal_time_col, 
            df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
        )
        df_mp_curr = mp_df_curr_hist['mp_df_curr']
        df_mp_hist = mp_df_curr_hist['mp_df_hist']
    #-------------------------
    # At a bare minimum, df_mp_curr and df_mp_hist must both have the following columns:
    #   necessary_mp_cols = ['mfr_devc_ser_nbr', 'prem_nb', 'inst_ts', 'rmvl_ts']
    assert(all([x in df_mp_curr.columns for x in necessary_mp_cols+[df_mp_trsf_pole_nb_col]]))
    assert(all([x in df_mp_hist.columns for x in necessary_mp_cols]))
    #-------------------------
    # PNs_for_xfmrs is a DF with trsf_pole_nbs indices and elements which are lists of PNs for each xfmr
    PNs_for_xfmrs = MeterPremise.get_SNs_andor_PNs_for_xfmrs(
        trsf_pole_nbs=trsf_pole_nbs, 
        include_SNs=False,
        include_PNs=True,
        trsf_pole_nb_col=df_mp_trsf_pole_nb_col, 
        serial_number_col=df_mp_serial_number_col, 
        prem_nb_col=df_mp_prem_nb_col, 
        return_SNs_col=None, #Not grabbing SNs
        return_PNs_col=return_prem_nbs_col, 
        assert_all_trsf_pole_nbs_found=assert_all_trsf_pole_nbs_found, 
        mp_df=df_mp_curr, 
        return_mp_df_also=False
    )
    #-------------------------
    # Join together rcpo_df, time_infos_df and PNs_for_xfmrs
    rcpo_df = merge_rcpo_and_df(
        rcpo_df=rcpo_df, 
        df_2=time_infos_df, 
        rcpo_df_on=rcpo_df_to_time_infos_on,
        df_2_on=time_infos_to_rcpo_df_on, 
        how=how
    )
    #-----
    rcpo_df = merge_rcpo_and_df(
        rcpo_df=rcpo_df, 
        df_2=PNs_for_xfmrs, 
        rcpo_df_on=rcpo_df_to_PNs_on,
        df_2_on=PNs_to_rcpo_df_on, 
        how=how
    )
    #--------------------------------------------------
    # Only reason for making dict is to ensure trsf_pole_nbs are not repeated 
    active_SNs_in_xfmrs_dfs_dict = {}

    rcpo_idx_names = list(rcpo_df.index.names)
    assert(not any([x is None for x in rcpo_idx_names]))
    for idx_i, row_i in rcpo_df.iterrows():
        # active_SNs_df_i will have indices equal to premise numbers and value equal to lists
        #   of active SNs for each PN
        # Purpose of making idx_names_w_vals a list of tuples, instead of a dict, is to ensure the correct order is maintained
        #   Dicts usually return the correct order, but this is not guaranteed
        if len(rcpo_idx_names)==1:
            assert(rcpo_df.index.nlevels==1)
            idx_names_w_vals = [(rcpo_idx_names[0], idx_i)]
        else:
            idx_names_w_vals = [((rcpo_idx_names[i] if i!=trsf_pole_nbs_idx_lvl else df_mp_trsf_pole_nb_col), idx_i[i]) 
                                for i in range(len(idx_i))]
        PNs_i=row_i[return_prem_nbs_col]
        dt_0_i=row_i[t_min_col]
        dt_1_i=row_i[t_max_col]
        #-----
        # See NOTEs above regarding t_min/t_max being empty
        # In such a case, it is simply impossibe (with the summary files currently generated) to access
        #   the date over which the data would have been run, if any events existed.
        #   In future versions, this information will be included in the summary files!
        # I don't want to completely exclude these (by e.g., setting dt_0_i=pd.Timestamp.min and 
        #   dt_1_i=pd.Timestamp.max), so I will simply include the meters which are active TODAY.
        # This obviously is not correct, but this occurrence is rare (only happening when every single meter
        #   on a transformer had no events during the time period) and this crude approximation will be fine.
        if Utilities.is_object_one_of_types(dt_0_i, [list, np.ndarray]):
            assert(len(dt_0_i)==0)
            # I believe if this happens for one it should happen for both...
            assert(Utilities.is_object_one_of_types(dt_1_i, [list, np.ndarray]) and len(dt_1_i)==0)
            dt_0_i=pd.Timestamp.today()
        if Utilities.is_object_one_of_types(dt_1_i, [list, np.ndarray]):
            assert(len(dt_1_i)==0)
            # I believe if this happens for one it should happen for both...
            # But, dt_0_i changed already above, so must check row_i[t_min_col] instead!
            assert(Utilities.is_object_one_of_types(row_i[t_min_col], [list, np.ndarray]) and len(row_i[t_min_col])==0)
            dt_1_i=pd.Timestamp.today()
        active_SNs_df_i = MeterPremise.get_active_SNs_for_PNs_at_datetime_interval(
            PNs=PNs_i,
            df_mp_curr=df_mp_curr, 
            df_mp_hist=df_mp_hist, 
            dt_0=dt_0_i,
            dt_1=dt_1_i,
            output_index=None,
            output_groupby=[df_mp_prem_nb_col], 
            include_prems_wo_active_SNs_when_groupby=True, 
            assert_all_PNs_found=False
        )
        active_SNs_df_i=active_SNs_df_i.reset_index()
        if active_SNs_df_i.shape[0]==0:
            active_SNs_df_i[df_mp_prem_nb_col] = np.nan
            active_SNs_df_i[df_mp_serial_number_col] = [[]] 
            for name,val in idx_names_w_vals:
                active_SNs_df_i[name] = val
            active_SNs_df_i = active_SNs_df_i.set_index([x[0] for x in idx_names_w_vals])
        else:
            for name,val in idx_names_w_vals:
                active_SNs_df_i[name] = val
            active_SNs_df_i = active_SNs_df_i.explode(df_mp_serial_number_col)
            active_SNs_df_i = Utilities_df.consolidate_df(
                df=active_SNs_df_i, 
                groupby_cols=[x[0] for x in idx_names_w_vals], 
                cols_shared_by_group=None, 
                cols_to_collect_in_lists=[df_mp_serial_number_col, df_mp_prem_nb_col], 
                include_groupby_cols_in_output_cols=False, 
                allow_duplicates_in_lists=False, 
                recover_uniqueness_violators=True, 
                rename_cols=None, 
                verbose=False
            )
        assert(idx_i not in active_SNs_in_xfmrs_dfs_dict)
        active_SNs_in_xfmrs_dfs_dict[idx_i] = active_SNs_df_i
    #-------------------------
    active_SNs_df = pd.concat(list(active_SNs_in_xfmrs_dfs_dict.values()))
    #-------------------------
    # Change [nan] entries to []
    found_nans_srs = active_SNs_df[df_mp_serial_number_col].apply(lambda x: len([ix for ix in x if not pd.isna(ix)]))==0
    if found_nans_srs.sum()>0:
        active_SNs_df.loc[found_nans_srs, df_mp_serial_number_col] = [[]]
    #-----
    found_nans_srs = active_SNs_df[df_mp_prem_nb_col].apply(lambda x: len([ix for ix in x if not pd.isna(ix)]))==0
    if found_nans_srs.sum()>0:
        active_SNs_df.loc[found_nans_srs, df_mp_prem_nb_col] = [[]]
    #-------------------------
    active_SNs_df = active_SNs_df.rename(columns={
        df_mp_prem_nb_col:return_prem_nbs_col, 
        df_mp_serial_number_col:return_SNs_col
    })
    #-------------------------
    return active_SNs_df

In [ ]:
# Replaces get_active_SNs_for_xfmrs_OLD, but should probably build get_active_SNs_for_xfmrs
#  which accepts a list of trsf_pole_nbs instead of rcpo_df, which this function can use
def get_active_SNs_for_xfmrs_in_rcpo_df_v2(
    rcpo_df, 
    trsf_pole_nbs_loc, 
    df_mp_curr, 
    df_mp_hist,
    time_infos_df, 
    rcpo_df_to_time_infos_on = [('index', 'outg_rec_nb')], 
    time_infos_to_rcpo_df_on = ['index'], 
    how='left', 
    rcpo_df_to_PNs_on = [('index', 'trsf_pole_nb')], 
    PNs_to_rcpo_df_on = ['index'], 
    addtnl_mp_df_curr_cols=None, 
    addtnl_mp_df_hist_cols=None, 
    return_SNs_col='SNs', 
    return_prem_nbs_col='prem_nbs', 
    assert_all_trsf_pole_nbs_found=True, 
    df_mp_serial_number_col='mfr_devc_ser_nbr', 
    df_mp_prem_nb_col='prem_nb', 
    df_mp_install_time_col='inst_ts', 
    df_mp_removal_time_col='rmvl_ts', 
    df_mp_trsf_pole_nb_col='trsf_pole_nb', 
    t_min_col='t_min', 
    t_max_col='t_max'
):
    r"""
    Difficulty is that default.meter_premise_hist does not have trsf_pole_nb field.
    Therefore, one must use default.meter_premise to find the premise numbers for xfrms in trsf_pole_nbs,
      then use those PNs to select the correct entries from default.meter_premise_hist.
    The trsf_pole_nbs should be contained in rcpo_df, and will be found using the trsf_pole_nbs_loc
      parameter described below.
      
    trsf_pole_nbs_loc:
        Directs where the transformer pole numbers are located
        This should identify an index (w/ level)
        Set equal to 'index' for normal DFs, or when trsf_pole_nbs are in level 0 of index.
        For a DF with MultiIndex index, there are two options:
            i.  Set equal to f'index_{idx_level}' for a DF with MutliIndex index, where idx_level
                is an int identifying the level in which the trsf_pole_nbs reside
            ii. Set equal to the tuple ('index', trsf_pole_nbs_idx_name), where trsf_pole_nbs_idx_name is
            the name of the index level in which the trsf_pole_nbs reside.

    If df_mp_curr OR df_mp_hist is not supplied, both will be built!
    
    addtnl_mp_df_curr_cols/addtnl_mp_df_hist_cols:
      Only used when df_mp_curr/df_mp_hist not supplied and therefore need to be built
      
    """
    #--------------------------------------------------
    assert(t_min_col in time_infos_df.columns and 
           t_max_col in time_infos_df.columns)
    time_infos_df = time_infos_df[[t_min_col, t_max_col]]
    #--------------------------------------------------
    # trsf_pole_nbs_loc can be a string or tuple/list
    # First, find trsf_pole_nbs and trsf_pole_nbs_idx_lvl
    assert(Utilities.is_object_one_of_types(trsf_pole_nbs_loc, [str, list, tuple]))
    if isinstance(trsf_pole_nbs_loc, str):
        assert(trsf_pole_nbs_loc.startswith('index'))
        if trsf_pole_nbs_loc=='index':
            trsf_pole_nbs_idx_lvl = 0
        else:
            trsf_pole_nbs_idx_lvl = re.findall('index_(\d*)', trsf_pole_nbs_loc)
            assert(len(trsf_pole_nbs_idx_lvl)==1)
            trsf_pole_nbs_idx_lvl=trsf_pole_nbs_idx_lvl[0]
            trsf_pole_nbs_idx_lvl=int(trsf_pole_nbs_idx_lvl)
    else:
        assert(len(trsf_pole_nbs_loc)==2)
        assert(trsf_pole_nbs_loc[0]=='index')
        assert(trsf_pole_nbs_loc[1] in rcpo_df.index.names)
        trsf_pole_nbs_idx_lvl = rcpo_df.index.names.index(trsf_pole_nbs_loc[1])
        #---------------
        assert(trsf_pole_nbs_idx_lvl < rcpo_df.index.nlevels)
        trsf_pole_nbs = rcpo_df.index.get_level_values(trsf_pole_nbs_idx_lvl).tolist()
    #--------------------------------------------------
    #-------------------------
    necessary_mp_cols = [df_mp_serial_number_col, df_mp_prem_nb_col, df_mp_install_time_col, df_mp_removal_time_col]
    #-------------------------
    if df_mp_curr is None or df_mp_hist is None:
        mp_df_curr_hist = MeterPremise.build_mp_df_curr_hist_for_xfmrs(
            trsf_pole_nbs, 
            join_curr_hist=False, 
            addtnl_mp_df_curr_cols=addtnl_mp_df_curr_cols, 
            addtnl_mp_df_hist_cols=addtnl_mp_df_hist_cols, 
            df_mp_serial_number_col=df_mp_serial_number_col, 
            df_mp_prem_nb_col=df_mp_prem_nb_col, 
            df_mp_install_time_col=df_mp_install_time_col, 
            df_mp_removal_time_col=df_mp_removal_time_col, 
            df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
        )
        df_mp_curr = mp_df_curr_hist['mp_df_curr']
        df_mp_hist = mp_df_curr_hist['mp_df_hist']
    #-------------------------
    # At a bare minimum, df_mp_curr and df_mp_hist must both have the following columns:
    #   necessary_mp_cols = ['mfr_devc_ser_nbr', 'prem_nb', 'inst_ts', 'rmvl_ts']
    assert(all([x in df_mp_curr.columns for x in necessary_mp_cols+[df_mp_trsf_pole_nb_col]]))
    assert(all([x in df_mp_hist.columns for x in necessary_mp_cols]))
    #-------------------------
    # PNs_for_xfmrs is a DF with trsf_pole_nbs indices and elements which are lists of PNs for each xfmr
    PNs_for_xfmrs = MeterPremise.get_SNs_andor_PNs_for_xfmrs(
        trsf_pole_nbs=trsf_pole_nbs, 
        include_SNs=False,
        include_PNs=True,
        trsf_pole_nb_col=df_mp_trsf_pole_nb_col, 
        serial_number_col=df_mp_serial_number_col, 
        prem_nb_col=df_mp_prem_nb_col, 
        return_SNs_col=None, #Not grabbing SNs
        return_PNs_col=return_prem_nbs_col, 
        assert_all_trsf_pole_nbs_found=assert_all_trsf_pole_nbs_found, 
        mp_df=df_mp_curr, 
        return_mp_df_also=False
    )
    #-------------------------
    # Join together rcpo_df, time_infos_df and PNs_for_xfmrs
    rcpo_df = merge_rcpo_and_df(
        rcpo_df=rcpo_df, 
        df_2=time_infos_df, 
        rcpo_df_on=rcpo_df_to_time_infos_on,
        df_2_on=time_infos_to_rcpo_df_on, 
        how=how
    )
    #-----
    rcpo_df = merge_rcpo_and_df(
        rcpo_df=rcpo_df, 
        df_2=PNs_for_xfmrs, 
        rcpo_df_on=rcpo_df_to_PNs_on,
        df_2_on=PNs_to_rcpo_df_on, 
        how=how
    )
    #--------------------------------------------------
    # Only reason for making dict is to ensure trsf_pole_nbs are not repeated 
    active_SNs_in_xfmrs_dfs_dict = {}

    rcpo_idx_names = list(rcpo_df.index.names)
    assert(not any([x is None for x in rcpo_idx_names]))
    for idx_i, row_i in rcpo_df.iterrows():
        # active_SNs_df_i will have indices equal to premise numbers and value equal to lists
        #   of active SNs for each PN
        # Purpose of making idx_names_w_vals a list of tuples, instead of a dict, is to ensure the correct order is maintained
        #   Dicts usually return the correct order, but this is not guaranteed
        if len(rcpo_idx_names)==1:
            assert(rcpo_df.index.nlevels==1)
            idx_names_w_vals = [(rcpo_idx_names[0], idx_i)]
        else:
            idx_names_w_vals = [((rcpo_idx_names[i] if i!=trsf_pole_nbs_idx_lvl else df_mp_trsf_pole_nb_col), idx_i[i]) 
                                for i in range(len(idx_i))]
        PNs_i=row_i[return_prem_nbs_col]
        dt_0_i=row_i[t_min_col]
        dt_1_i=row_i[t_max_col]
        #-----
        # See NOTEs above regarding t_min/t_max being empty
        # In such a case, it is simply impossibe (with the summary files currently generated) to access
        #   the date over which the data would have been run, if any events existed.
        #   In future versions, this information will be included in the summary files!
        # I don't want to completely exclude these (by e.g., setting dt_0_i=pd.Timestamp.min and 
        #   dt_1_i=pd.Timestamp.max), so I will simply include the meters which are active TODAY.
        # This obviously is not correct, but this occurrence is rare (only happening when every single meter
        #   on a transformer had no events during the time period) and this crude approximation will be fine.
        if Utilities.is_object_one_of_types(dt_0_i, [list, np.ndarray]):
            assert(len(dt_0_i)==0)
            # I believe if this happens for one it should happen for both...
            assert(Utilities.is_object_one_of_types(dt_1_i, [list, np.ndarray]) and len(dt_1_i)==0)
            dt_0_i=pd.Timestamp.today()
        if Utilities.is_object_one_of_types(dt_1_i, [list, np.ndarray]):
            assert(len(dt_1_i)==0)
            # I believe if this happens for one it should happen for both...
            # But, dt_0_i changed already above, so must check row_i[t_min_col] instead!
            assert(Utilities.is_object_one_of_types(row_i[t_min_col], [list, np.ndarray]) and len(row_i[t_min_col])==0)
            dt_1_i=pd.Timestamp.today()
        active_SNs_df_i = MeterPremise.get_active_SNs_for_PNs_at_datetime_interval(
            PNs=PNs_i,
            df_mp_curr=df_mp_curr, 
            df_mp_hist=df_mp_hist, 
            dt_0=dt_0_i,
            dt_1=dt_1_i,
            output_index=None,
            output_groupby=[df_mp_prem_nb_col], 
            include_prems_wo_active_SNs_when_groupby=True, 
            assert_all_PNs_found=False
        )
        active_SNs_df_i=active_SNs_df_i.reset_index()
        if active_SNs_df_i.shape[0]==0:
            active_SNs_df_i[df_mp_prem_nb_col] = np.nan
            active_SNs_df_i[df_mp_serial_number_col] = [[]] 
        for name,val in idx_names_w_vals:
            active_SNs_df_i[name] = val
        active_SNs_df_i = active_SNs_df_i.explode(df_mp_serial_number_col)
        assert(idx_i not in active_SNs_in_xfmrs_dfs_dict)
        active_SNs_in_xfmrs_dfs_dict[idx_i] = active_SNs_df_i
    #-------------------------
    active_SNs_df = pd.concat(list(active_SNs_in_xfmrs_dfs_dict.values()))
    #-------------------------
    active_SNs_df = Utilities_df.consolidate_df(
        df=active_SNs_df, 
        groupby_cols=[x[0] for x in idx_names_w_vals], 
        cols_shared_by_group=None, 
        cols_to_collect_in_lists=[df_mp_serial_number_col, df_mp_prem_nb_col], 
        include_groupby_cols_in_output_cols=False, 
        allow_duplicates_in_lists=False, 
        recover_uniqueness_violators=True, 
        rename_cols=None, 
        verbose=False
    )
    #-------------------------
    # Change [nan] entries to []
    found_nans_srs = active_SNs_df[df_mp_serial_number_col].apply(lambda x: len([ix for ix in x if not pd.isna(ix)]))==0
    if found_nans_srs.sum()>0:
        active_SNs_df.loc[found_nans_srs, df_mp_serial_number_col] = [[]]
    #-----
    found_nans_srs = active_SNs_df[df_mp_prem_nb_col].apply(lambda x: len([ix for ix in x if not pd.isna(ix)]))==0
    if found_nans_srs.sum()>0:
        active_SNs_df.loc[found_nans_srs, df_mp_prem_nb_col] = [[]]
    #-------------------------
    active_SNs_df = active_SNs_df.rename(columns={
        df_mp_prem_nb_col:return_prem_nbs_col, 
        df_mp_serial_number_col:return_SNs_col
    })
    #-------------------------
    return active_SNs_df

In [ ]:
# Replaces add_xfmr_active_SNs_to_rcpo_df_OLD
def add_xfmr_active_SNs_to_rcpo_df(
    rcpo_df, 
    trsf_pole_nbs_loc, 
    set_xfmr_nSNs=True, 
    include_active_xfmr_PNs=False, 
    df_mp_curr=None,
    df_mp_hist=None, 
    time_infos_df=None, 
    rcpo_df_to_time_infos_on = [('index', 'outg_rec_nb')], 
    time_infos_to_rcpo_df_on = ['index'], 
    how='left', 
    rcpo_df_to_PNs_on = [('index', 'trsf_pole_nb')], 
    PNs_to_rcpo_df_on = ['index'], 
    addtnl_get_active_SNs_for_xfmrs_kwargs=None, 
    xfmr_SNs_col='_xfmr_SNs', 
    xfmr_nSNs_col='_xfmr_nSNs', 
    xfmr_PNs_col='_xfmr_PNs', 
    xfmr_nPNs_col='_xfmr_nPNs', 
):
    r"""
    NOTE: If include_active_xfmr_PNs is True, this column (named xfmr_PNs_col='_xfmr_SNs') should be 
          equal to the PNs already in rcpo_df!
    NOTE: xfmr_SNs_col, xfmr_nSNs_col, xfmr_PNs_col, and xfmr_nPNs_col should all be strings, not tuples.
          If column is multiindex, the level_0 value will be handled below.
          
    NOTE: If any of xfmr_SNs_col, xfmr_nSNs_col, xfmr_PNs_col, and xfmr_nPNs_col are already contained in 
          rcpo_df, they will be replaced.  This is needed so that the merge operation does not come back with _x and _y
          values.  So, one should make sure this function call is truly needed, as grabbing the serial numbers for the
          outages typically takes a couple/few minutes.
          
    NOTE: To make things run faster, the user can supply df_mp_curr and df_mp_hist.  These will be included in 
          get_active_SNs_for_xfmrs_kwargs.
          NOTE: If df_mp_curr/df_mp_hist is also supplied in addtnl_get_active_SNs_for_xfmrs_kwargs,
                that/those in addtnl_get_active_SNs_for_xfmrs_kwargs will ultimately be used (not the
                explicity df_mp_hist/curr in the function arguments!)
          CAREFUL: If one does supple df_mp_curr/hist, one must be certain these DFs contain all necessary elements!
    """
    #-------------------------
    get_active_SNs_for_xfmrs_kwargs = dict(
        rcpo_df=rcpo_df, 
        trsf_pole_nbs_loc=trsf_pole_nbs_loc, 
        df_mp_curr=df_mp_curr, 
        df_mp_hist=df_mp_hist, 
        time_infos_df=time_infos_df, 
        rcpo_df_to_time_infos_on=rcpo_df_to_time_infos_on, 
        time_infos_to_rcpo_df_on=time_infos_to_rcpo_df_on, 
        how=how, 
        rcpo_df_to_PNs_on=rcpo_df_to_PNs_on, 
        PNs_to_rcpo_df_on=PNs_to_rcpo_df_on, 
        return_prem_nbs_col=xfmr_PNs_col, 
        return_SNs_col=xfmr_SNs_col
    )
    if addtnl_get_active_SNs_for_xfmrs_kwargs is not None:
        get_active_SNs_for_xfmrs_kwargs = {**get_active_SNs_for_xfmrs_kwargs, 
                                           **addtnl_get_active_SNs_for_xfmrs_kwargs}
    active_SNs_df = get_active_SNs_for_xfmrs_in_rcpo_df(**get_active_SNs_for_xfmrs_kwargs)
    assert(isinstance(active_SNs_df, pd.DataFrame))
    #-------------------------
    # Assert below might be too strong here...
    assert(sorted(rcpo_df.index.unique().tolist())==sorted(active_SNs_df.index.unique().tolist()))
    assert(rcpo_df.columns.nlevels<=2)
    if rcpo_df.columns.nlevels==1:
        #----------
        # See note above about columns being replaced/dropped
        cols_to_drop = [x for x in rcpo_df.columns if x in active_SNs_df.columns]
        if len(cols_to_drop)>0:
            rcpo_df = rcpo_df.drop(columns=cols_to_drop)
        #----------
        rcpo_df = rcpo_df.merge(active_SNs_df, left_index=True, right_index=True)
        #----------
        if set_xfmr_nSNs:
            rcpo_df = MECPODf.set_nSNs_from_SNs_in_rcpo_df(rcpo_df, xfmr_SNs_col, xfmr_nSNs_col)
            if include_active_xfmr_PNs:
                rcpo_df = MECPODf.set_nSNs_from_SNs_in_rcpo_df(rcpo_df, xfmr_PNs_col, xfmr_nPNs_col)
    else:
        # Currently, only expecting raw and/or norm.  No problem to allow more, but for now keep this to alert 
        # of anything unexpected
        assert(rcpo_df.columns.get_level_values(0).nunique()<=2)
        for i,level_0_val in enumerate(rcpo_df.columns.get_level_values(0).unique()):
            if i==0:
                active_SNs_df.columns = pd.MultiIndex.from_product([[level_0_val], active_SNs_df.columns])
            else:
                active_SNs_df.columns = active_SNs_df.columns.set_levels([level_0_val], level=0)
            #----------
            # See note above about columns being replaced/dropped
            cols_to_drop = [x for x in rcpo_df.columns if x in active_SNs_df.columns]
            if len(cols_to_drop)>0:
                rcpo_df = rcpo_df.drop(columns=cols_to_drop)
            #----------
            rcpo_df = rcpo_df.merge(active_SNs_df, left_index=True, right_index=True)
            #----------
            if set_xfmr_nSNs:
                rcpo_df = MECPODf.set_nSNs_from_SNs_in_rcpo_df(rcpo_df, (level_0_val, xfmr_SNs_col), (level_0_val, xfmr_nSNs_col))
                if include_active_xfmr_PNs:
                    rcpo_df = MECPODf.set_nSNs_from_SNs_in_rcpo_df(rcpo_df, (level_0_val, xfmr_PNs_col), (level_0_val, xfmr_nPNs_col))
    #-------------------------
    rcpo_df = rcpo_df.sort_index(axis=1,level=0)
    #-------------------------
    return rcpo_df

In [ ]:
# Replaces build_rcpo_df_norm_by_xfmr_active_nSNs_OLD
def build_rcpo_df_norm_by_xfmr_active_nSNs(
    rcpo_df_raw, 
    trsf_pole_nbs_loc, 
    xfmr_nSNs_col='_xfmr_nSNs', 
    xfmr_SNs_col='_xfmr_SNs', 
    other_SNs_col_tags_to_ignore=['_SNs', '_nSNs', '_prem_nbs', '_nprem_nbs', '_xfmr_PNs', '_xfmr_nPNs'], 
    drop_xfmr_nSNs_eq_0=True, 
    new_level_0_val='counts_norm_by_xfmr_nSNs', 
    remove_SNs_cols=False, 
    df_mp_curr=None,
    df_mp_hist=None, 
    time_infos_df=None,
    rcpo_df_to_time_infos_on = [('index', 'outg_rec_nb')], 
    time_infos_to_rcpo_df_on = ['index'], 
    how='left', 
    rcpo_df_to_PNs_on = [('index', 'trsf_pole_nb')], 
    PNs_to_rcpo_df_on = ['index'], 
    addtnl_get_active_SNs_for_xfmrs_kwargs=None
):
    r"""
    Build rcpo_df normalized by the number of serial numbers in each outage

    drop_xfmr_nSNs_eq_0:
      It is possible for the number of serial numbers in an outage to be zero!
      Premise numbers are always found, but the meter_premise database does not always 
        contain the premise numbers.
      Dividing by zero will make all counts for such an entry equal to NaN or inf.
      When drop_xfmr_nSNs_eq_0 is True, such entries will be removed.

    NOTE: xfmr_SNs_col and xfmr_nSNs_col should both be strings, not tuples.
          If column is MultiIndex, the level_0 value will be handled below.
    """
    #-------------------------
    n_counts_col = xfmr_nSNs_col
    list_col = xfmr_SNs_col
    #-------------------------
    # NOTE: MECPODf.add_outage_SNs_to_rcpo_df expects xfmr_SNs_col and xfmr_nSNs_col to be strings, not tuples
    #       as it handles the level 0 values if they exist.  So, if tuples, use only highest level values (i.e., level 1)
    assert(Utilities.is_object_one_of_types(list_col, [str, tuple]))
    assert(Utilities.is_object_one_of_types(n_counts_col, [str, tuple]))
    #-----
    add_list_col_to_rcpo_df_func = add_xfmr_active_SNs_to_rcpo_df
    add_list_col_to_rcpo_df_kwargs = dict(
        trsf_pole_nbs_loc=trsf_pole_nbs_loc, 
        set_xfmr_nSNs=True, 
        include_active_xfmr_PNs=True, 
        df_mp_curr=df_mp_curr,
        df_mp_hist=df_mp_hist, 
        time_infos_df=time_infos_df, 
        rcpo_df_to_time_infos_on=rcpo_df_to_time_infos_on, 
        time_infos_to_rcpo_df_on=time_infos_to_rcpo_df_on, 
        how=how, 
        rcpo_df_to_PNs_on=rcpo_df_to_PNs_on, 
        PNs_to_rcpo_df_on=PNs_to_rcpo_df_on, 
        addtnl_get_active_SNs_for_xfmrs_kwargs=addtnl_get_active_SNs_for_xfmrs_kwargs, 
        xfmr_SNs_col='_xfmr_SNs', 
        xfmr_nSNs_col='_xfmr_nSNs', 
        xfmr_PNs_col='_xfmr_PNs', 
        xfmr_nPNs_col='_xfmr_nPNs', 
    )
    #-------------------------
    other_col_tags_to_ignore = other_SNs_col_tags_to_ignore
    drop_n_counts_eq_0 = drop_xfmr_nSNs_eq_0
    new_level_0_val = new_level_0_val
    remove_ignored_cols = remove_SNs_cols
    #-------------------------
    return MECPODf.build_rcpo_df_norm_by_list_counts(
        rcpo_df_raw=rcpo_df_raw, 
        n_counts_col=n_counts_col, 
        list_col=list_col, 
        add_list_col_to_rcpo_df_func=add_list_col_to_rcpo_df_func, 
        add_list_col_to_rcpo_df_kwargs=add_list_col_to_rcpo_df_kwargs, 
        other_col_tags_to_ignore=other_col_tags_to_ignore, 
        drop_n_counts_eq_0=drop_n_counts_eq_0, 
        new_level_0_val=new_level_0_val, 
        remove_ignored_cols=remove_ignored_cols
    )

# =============================================================
# =============================================================

In [ ]:
run_testing_data=True
fig_num=0

dovs_and_end_events_data_dir = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data'
if run_testing_data:
    dovs_and_end_events_data_dir = os.path.join(dovs_and_end_events_data_dir, r'TESTING_DATASETS')

files_dir_outg             = os.path.join(dovs_and_end_events_data_dir, r'EndEvents')
files_dir_outg_prim_strict = os.path.join(dovs_and_end_events_data_dir, r'EndEvents_prim_strict')
files_dir_no_outg = os.path.join(dovs_and_end_events_data_dir, r'EndEvents_NoOutg')

file_path_glob = r'end_events_[0-9]*.csv'
file_path_regex = None


assert_all_cols_equal=True
include_normalize_by_nSNs=True
inclue_zero_counts=True
return_multiindex_outg_reason=False
return_normalized_separately=False

## No Outage

In [ ]:
# NO OUTG
outg_rec_nb_col='trsf_pole_nb'
group_cols='trsf_pole_nb'
addtnl_dropna_subset_cols=None

prem_nb_col='aep_premise_nb'
#-------------------------
paths_no_outg = Utilities.find_all_paths(
    base_dir=files_dir_no_outg, 
    glob_pattern=file_path_glob, 
    regex_pattern=file_path_regex
)
paths_no_outg = paths_no_outg[0:10]
#-------------------------
end_events_df_no_outg = GenAn.read_df_from_csv_batch(
    paths=paths_no_outg, 
    cols_and_types_to_convert_dict=None, 
    to_numeric_errors='coerce', 
    make_all_columns_lowercase=True, 
    assert_all_cols_equal=True
)
#-------------------------
if (f'{outg_rec_nb_col}_gpd_for_sql' in end_events_df_no_outg.columns and 
    outg_rec_nb_col not in end_events_df_no_outg.columns):
    end_events_df_no_outg = end_events_df_no_outg.rename(columns={f'{outg_rec_nb_col}_gpd_for_sql':outg_rec_nb_col})
assert(outg_rec_nb_col in end_events_df_no_outg.columns)
#-------------------------
dropna_subset_cols = [outg_rec_nb_col]
if addtnl_dropna_subset_cols is not None:
    dropna_subset_cols.extend(addtnl_dropna_subset_cols)
end_events_df_no_outg = end_events_df_no_outg.dropna(subset=dropna_subset_cols)
#-------------------------
end_events_df_no_outg = AMIEndEvents.reduce_end_event_reasons_in_df(
    df=end_events_df_no_outg, 
    patterns_to_replace=patterns_to_replace
)
#-------------------------
rcpo_df_no_outg = AMIEndEvents.get_reason_counts_per_group(
    end_events_df_no_outg, 
    group_cols=group_cols,
    group_freq=None, 
    serial_number_col='serialnumber', 
    reason_col='reason', 
    include_normalize_by_nSNs=False, 
    inclue_zero_counts=False,
    possible_reasons=None, 
    include_nSNs=True, 
    include_SNs=True, 
    prem_nb_col='aep_premise_nb', 
    include_nprem_nbs=False,
    include_prem_nbs=False, 
    return_form = dict(return_multiindex_outg_reason = False, 
                       return_normalized_separately  = False)    
)

In [ ]:
rcpo_df_no_outg.head()

In [ ]:
rcpo_df_no_outg.shape

In [ ]:
# TODO This has two trsf_pole_nbs, should drop one
end_events_df_no_outg['trsf_pole_nb'].equals(end_events_df_no_outg['trsf_pole_nb_gpd_for_sql'])

In [ ]:
# Build no_outg_time_infos_df, which has prem_nbs indices and t_min, t_max (and possible summary_path) columns
# This is where the time information for each premise number comes from
no_outg_time_infos_df = MECPOAn.get_bsln_time_interval_infos_df_from_summary_files(
    summary_paths=[AMIEndEvents.find_summary_file_from_csv(x) for x in paths_no_outg], 
    output_prem_nbs_col='prem_nbs', 
    output_t_min_col='t_min', 
    output_t_max_col='t_max', 
    make_prem_nbs_idx=True, 
    include_summary_paths=True
)

In [ ]:
# mp_df_curr_hist_no_outgs = MeterPremise.build_mp_df_curr_hist_for_xfmrs(trsf_pole_nbs=rcpo_df_1.index.tolist())
mp_df_curr_hist_no_outgs = MeterPremise.build_mp_df_curr_hist_for_xfmrs(trsf_pole_nbs=rcpo_df_no_outg.index.get_level_values(0).tolist())
print(mp_df_curr_hist_no_outgs['mp_df_curr'].shape)
print(mp_df_curr_hist_no_outgs['mp_df_hist'].shape)

In [ ]:
active_SNs_for_xfmrs_no_outg = get_active_SNs_for_xfmrs_OLD(
    trsf_pole_nbs=rcpo_df_no_outg.index.tolist(), 
    df_mp_curr=mp_df_curr_hist_no_outgs['mp_df_curr'], 
    df_mp_hist=mp_df_curr_hist_no_outgs['mp_df_hist'],
    no_outg_time_infos_df=no_outg_time_infos_df
)

In [ ]:
active_SNs_for_xfmrs_no_outg

In [ ]:
# rcpo_df_no_outg = add_xfmr_active_SNs_to_rcpo_df_OLD(
#     rcpo_df_no_outg.copy(), 
#     df_mp_curr=mp_df_curr_hist_no_outgs['mp_df_curr'], 
#     df_mp_hist=mp_df_curr_hist_no_outgs['mp_df_hist'],
#     no_outg_time_infos_df=no_outg_time_infos_df   
# )

rcpo_df_no_outg = build_rcpo_df_norm_by_xfmr_active_nSNs_OLD(
    rcpo_df_no_outg.copy(), 
    df_mp_curr=mp_df_curr_hist_no_outgs['mp_df_curr'], 
    df_mp_hist=mp_df_curr_hist_no_outgs['mp_df_hist'],
    no_outg_time_infos_df=no_outg_time_infos_df   
)

In [ ]:
# trsf_pole_nbs=rcpo_df_no_outg.index.tolist()
# df_mp_curr=mp_df_curr_hist_no_outgs['mp_df_curr']
# df_mp_hist=mp_df_curr_hist_no_outgs['mp_df_hist'] 
# no_outg_time_infos_df=no_outg_time_infos_df
# addtnl_mp_df_curr_cols=None
# addtnl_mp_df_hist_cols=None 
# # files_dir_no_outg=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\EndEvents_NoOutg'
# file_path_glob_no_outg = r'end_events_[0-9]*.csv'
# return_SNs_col='SNs'
# return_prem_nbs_col='prem_nbs' 
# assert_all_trsf_pole_nbs_found=True 
# df_mp_serial_number_col='mfr_devc_ser_nbr' 
# df_mp_prem_nb_col='prem_nb'
# df_mp_install_time_col='inst_ts' 
# df_mp_removal_time_col='rmvl_ts' 
# df_mp_trsf_pole_nb_col='trsf_pole_nb' 
# t_min_col='t_min'
# t_max_col='t_max'
# #-------------------------
# necessary_mp_cols = [df_mp_serial_number_col, df_mp_prem_nb_col, df_mp_install_time_col, df_mp_removal_time_col]
# #-------------------------
# # At a bare minimum, df_mp_curr and df_mp_hist must both have the following columns:
# #   necessary_mp_cols = ['mfr_devc_ser_nbr', 'prem_nb', 'inst_ts', 'rmvl_ts']
# assert(all([x in df_mp_curr.columns for x in necessary_mp_cols+[df_mp_trsf_pole_nb_col]]))
# assert(all([x in df_mp_hist.columns for x in necessary_mp_cols]))
# #-------------------------
# # PNs_for_xfmrs is a DF with trsf_pole_nbs indices and elements which are lists of PNs for each xfmr
# PNs_for_xfmrs = MeterPremise.get_SNs_andor_PNs_for_xfmrs(
#     trsf_pole_nbs=trsf_pole_nbs, 
#     include_SNs=False,
#     include_PNs=True,
#     trsf_pole_nb_col=df_mp_trsf_pole_nb_col, 
#     serial_number_col=df_mp_serial_number_col, 
#     prem_nb_col=df_mp_prem_nb_col, 
#     return_SNs_col=None, #Not grabbing SNs
#     return_PNs_col=return_prem_nbs_col, 
#     assert_all_trsf_pole_nbs_found=assert_all_trsf_pole_nbs_found, 
#     mp_df=df_mp_curr, 
#     return_mp_df_also=False
# )
# #-------------------------
# # Instead of a DF with trsf_pole_nb index and prem_nb column, we want opposite
# xfmr_for_PNs_df = PNs_for_xfmrs.explode(return_prem_nbs_col)
# xfmr_for_PNs_df[xfmr_for_PNs_df.index.name] = xfmr_for_PNs_df.index
# xfmr_for_PNs_df = xfmr_for_PNs_df.set_index(return_prem_nbs_col)  
# #-------------------------
# #-------------------------
# # Merge xfmr_for_PNs_df with no_outg_time_infos_df to append the time data to the former
# # NOTE: It is possible for t_min/t_max to be NaT (NaN) for some entries after the merge, meaning that the 
# #       premise numbers were not found in no_outg_time_infos_df
# #       This happens because these premise numbers must not have had any meter events, and thus were not included 
# #         in the SQL query (as it takes a long time to find empty results, so I weed these out before running the 
# #         query), and therefore the premise numbers were not found in the summary files/no_outg_time_infos_df.    
# xfmr_for_PNs_df = pd.merge(xfmr_for_PNs_df, no_outg_time_infos_df, how='left', left_index=True, right_index=True)
# #-------------------------
# # Want to consolidate xfmr_for_PNs_df, grouping by trsf_pole_nb and collecting t_min,t_max, and a list
# # of the premise numbers.  Therefore, first the index must be reset to make a PNs columns
# xfmr_for_PNs_df=xfmr_for_PNs_df.reset_index()
# #-----
# # # Consolidate xfmr_for_PNs_df
# # # NOTE: If t_min/t_max is NaT (NaN) for all premise numbers in a given trsf_pole_nb (see NOTE above before merge
# # #       with no_outg_time_infos_df), then Utilities_df.consolidate_df will return an empty list (technically, an
# # #       empty np.ndarray) for that trsf_pole_nb
# # xfmr_for_PNs_df=Utilities_df.consolidate_df(
# #     df=xfmr_for_PNs_df, 
# #     groupby_col=df_mp_trsf_pole_nb_col, 
# #     cols_shared_by_group=[t_min_col, t_max_col], 
# #     cols_to_collect_in_lists=[return_prem_nbs_col], 
# #     verbose=True
# # )   
# #-------------------------

In [ ]:
# df = xfmr_for_PNs_df.copy()
# groupby_col=df_mp_trsf_pole_nb_col
# cols_shared_by_group=[t_min_col, t_max_col]
# cols_to_collect_in_lists=[return_prem_nbs_col]
# include_groupby_col_in_output_cols=False
# allow_duplicates_in_lists=False
# recover_uniqueness_violators=True
# rename_cols=None
# verbose=False
# #-------------------------
# if cols_shared_by_group is None or len(cols_shared_by_group)==0:
#     cols_shared_by_group = groupby_col
# assert(Utilities.is_object_one_of_types(cols_shared_by_group, [list, str, int]))
# if not isinstance(cols_shared_by_group, list):
#     cols_shared_by_group=[cols_shared_by_group]
# #-----
# assert(Utilities.is_object_one_of_types(cols_to_collect_in_lists, [list, str, int]))
# if not isinstance(cols_to_collect_in_lists, list):
#     cols_to_collect_in_lists=[cols_to_collect_in_lists]
# #-------------------------
# # Only include groups which have one unique entry for each of cols_shared_by_group
# # The below series has a True/False value for each group stating whether (True)
# #   or not (False) all cols in cols_shared_by_group have one unique value
# group_has_unq_val_where_expected = (df.groupby(groupby_col)[cols_shared_by_group].nunique()<=1).all(axis=1)

# # Those not satisfying the uniqueness criterion are collected below
# groups_violating_uniqueness = group_has_unq_val_where_expected[~group_has_unq_val_where_expected.values].index.tolist()
# #-------------------------
# red_df = df.groupby(groupby_col).filter(lambda x: x.name not in groups_violating_uniqueness)
# #-------------------------
# # Consolidate xfmr_for_PNs_df
# # NOTE: If t_min/t_max is NaT (NaN) for all premise numbers in a given trsf_pole_nb (see NOTE above before merge
# #       with no_outg_time_infos_df), then Utilities_df.consolidate_df will return an empty list (technically, an
# #       empty np.ndarray) for that trsf_pole_nb
# xfmr_for_PNs_df=Utilities_df.consolidate_df_OLD(
#     df=xfmr_for_PNs_df, 
#     groupby_col=df_mp_trsf_pole_nb_col, 
#     cols_shared_by_group=[t_min_col, t_max_col], 
#     cols_to_collect_in_lists=[return_prem_nbs_col], 
#     verbose=True
# )

In [ ]:
# PNs_for_xfmrs_no_outg   = PNs_for_xfmrs.copy()
# xfmr_for_PNs_df_no_outg = xfmr_for_PNs_df.copy()

In [ ]:
# xfmr_for_PNs_df_no_outg

## Outage

In [ ]:
#OUTG
outg_rec_nb_col='outg_rec_nb'
# group_cols='outg_rec_nb'
group_cols=['outg_rec_nb', 'trsf_pole_nb']
addtnl_dropna_subset_cols=None

prem_nb_col='aep_premise_nb'
#-------------------------
paths_outg = Utilities.find_all_paths(
    base_dir=files_dir_outg, 
    glob_pattern=file_path_glob, 
    regex_pattern=file_path_regex
)
paths_outg = paths_outg[:10]
#-------------------------
end_events_df_outg = GenAn.read_df_from_csv_batch(
    paths=paths_outg, 
    cols_and_types_to_convert_dict=None, 
    to_numeric_errors='coerce', 
    make_all_columns_lowercase=True, 
    assert_all_cols_equal=True
)
#-------------------------
if (f'{outg_rec_nb_col}_gpd_for_sql' in end_events_df_outg.columns and 
    outg_rec_nb_col not in end_events_df_outg.columns):
    end_events_df_outg = end_events_df_outg.rename(columns={f'{outg_rec_nb_col}_gpd_for_sql':outg_rec_nb_col})
assert(outg_rec_nb_col in end_events_df_outg.columns)
#-------------------------
dropna_subset_cols = [outg_rec_nb_col]
if addtnl_dropna_subset_cols is not None:
    dropna_subset_cols.extend(addtnl_dropna_subset_cols)
end_events_df_outg = end_events_df_outg.dropna(subset=dropna_subset_cols)
#-------------------------
end_events_df_outg = AMIEndEvents.reduce_end_event_reasons_in_df(
    df=end_events_df_outg, 
    patterns_to_replace=patterns_to_replace
)
#-------------------------
rcpo_df_outg = AMIEndEvents.get_reason_counts_per_group(
    end_events_df_outg, 
    group_cols=group_cols,
    group_freq=None, 
    serial_number_col='serialnumber', 
    reason_col='reason', 
    include_normalize_by_nSNs=False, 
    inclue_zero_counts=False,
    possible_reasons=None, 
    include_nSNs=True, 
    include_SNs=True, 
    prem_nb_col='aep_premise_nb', 
    include_nprem_nbs=False,
    include_prem_nbs=False, 
    return_form = dict(return_multiindex_outg_reason = False, 
                       return_normalized_separately  = False)    
)
rcpo_df_outg

In [ ]:
group_cols

In [ ]:
rcpo_df = rcpo_df_outg.copy()

In [ ]:
dovs_outgs = DOVSOutages(                 
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True, 
    build_sql_function=DOVSOutages_SQL.build_sql_outage, 
    build_sql_function_kwargs=dict(
        outg_rec_nbs=rcpo_df.index.get_level_values(0).tolist(), 
        from_table_alias='DOV', 
        datetime_col='DT_OFF_TS_FULL', 
        cols_of_interest=[
            'OUTG_REC_NB', 
            dict(field_desc=f"DOV.DT_ON_TS - DOV.STEP_DRTN_NB/(60*24)", 
                 alias='DT_OFF_TS_FULL', table_alias_prefix=None)
        ], 
        field_to_split='outg_rec_nbs'
    ),
)
outg_dt_off_df = dovs_outgs.df
outg_dt_off_df = Utilities_df.convert_col_type(df=outg_dt_off_df, column='OUTG_REC_NB', to_type=str)
outg_dt_off_df=outg_dt_off_df.set_index('OUTG_REC_NB')
outg_dt_off_series = outg_dt_off_df['DT_OFF_TS_FULL']

#--------------------------------------------------
time_infos_df = outg_dt_off_df.copy()
time_infos_df['t_min'] = pd.to_datetime(time_infos_df['DT_OFF_TS_FULL'])-datetime.timedelta(days=1)
time_infos_df['t_max'] = pd.to_datetime(time_infos_df['DT_OFF_TS_FULL'])+datetime.timedelta(days=1)
time_infos_df = time_infos_df.drop(columns=['DT_OFF_TS_FULL'])

In [ ]:
time_infos_df

In [ ]:
# TODO NEED TO BE AUTOMATED
outg_rec_nb_idx_lvl=0
trsf_pole_nbs_idx_lvl=1

In [ ]:
if group_cols=='outg_rec_nb':
    mp_df_curr_hist_outg = DOVSOutages.build_mp_df_curr_hist_for_outgs(outg_rec_nbs=rcpo_df.index.get_level_values(outg_rec_nb_idx_lvl).tolist())
else:
    mp_df_curr_hist_outg = MeterPremise.build_mp_df_curr_hist_for_xfmrs(trsf_pole_nbs=rcpo_df.index.get_level_values(trsf_pole_nbs_idx_lvl).tolist())

In [ ]:
df_mp_curr=mp_df_curr_hist_outg['mp_df_curr']
df_mp_hist=mp_df_curr_hist_outg['mp_df_hist']

In [ ]:
rcpo_df

In [ ]:
# trsf_pole_nbs_loc = 'index_1'
trsf_pole_nbs_loc = ('index', 'trsf_pole_nb')
# trsf_pole_nbs_loc = 'index'
# trsf_pole_nbs_loc = ('counts', 'trsf_pole_nb')

# rcpo_df_to_time_infos_on = ['index_0']
rcpo_df_to_time_infos_on = [('index', 'outg_rec_nb')]
time_infos_to_rcpo_df_on = ['index']
how='left'


rcpo_df_to_PNs_on = [('index', 'trsf_pole_nb')]
PNs_to_rcpo_df_on = ['index']
how='left'

In [ ]:
return_SNs_col='SNs' 
return_prem_nbs_col='prem_nbs'
assert_all_trsf_pole_nbs_found=True
df_mp_serial_number_col='mfr_devc_ser_nbr'
df_mp_prem_nb_col='prem_nb'
df_mp_install_time_col='inst_ts'
df_mp_removal_time_col='rmvl_ts'
df_mp_trsf_pole_nb_col='trsf_pole_nb'
t_min_col='t_min'
t_max_col='t_max'

In [ ]:
start=time.time()
active_SNs_df = get_active_SNs_for_xfmrs_in_rcpo_df(
    rcpo_df=rcpo_df, 
    trsf_pole_nbs_loc=trsf_pole_nbs_loc, 
    df_mp_curr=df_mp_curr, 
    df_mp_hist=df_mp_hist,
    time_infos_df=time_infos_df, 
    rcpo_df_to_time_infos_on = rcpo_df_to_time_infos_on, 
    time_infos_to_rcpo_df_on = time_infos_to_rcpo_df_on, 
    how=how, 
    rcpo_df_to_PNs_on = rcpo_df_to_PNs_on, 
    PNs_to_rcpo_df_on = PNs_to_rcpo_df_on, 
    addtnl_mp_df_curr_cols=None, 
    addtnl_mp_df_hist_cols=None, 
    return_SNs_col='SNs', 
    return_prem_nbs_col='prem_nbs', 
    assert_all_trsf_pole_nbs_found=True, 
    df_mp_serial_number_col='mfr_devc_ser_nbr', 
    df_mp_prem_nb_col='prem_nb', 
    df_mp_install_time_col='inst_ts', 
    df_mp_removal_time_col='rmvl_ts', 
    df_mp_trsf_pole_nb_col='trsf_pole_nb', 
    t_min_col='t_min', 
    t_max_col='t_max'
)
print(time.time()-start)

In [ ]:
start=time.time()
active_SNs_df_2 = get_active_SNs_for_xfmrs_in_rcpo_df_v2(
    rcpo_df=rcpo_df, 
    trsf_pole_nbs_loc=trsf_pole_nbs_loc, 
    df_mp_curr=df_mp_curr, 
    df_mp_hist=df_mp_hist,
    time_infos_df=time_infos_df, 
    rcpo_df_to_time_infos_on = rcpo_df_to_time_infos_on, 
    time_infos_to_rcpo_df_on = time_infos_to_rcpo_df_on, 
    how=how, 
    rcpo_df_to_PNs_on = rcpo_df_to_PNs_on, 
    PNs_to_rcpo_df_on = PNs_to_rcpo_df_on, 
    addtnl_mp_df_curr_cols=None, 
    addtnl_mp_df_hist_cols=None, 
    return_SNs_col='SNs', 
    return_prem_nbs_col='prem_nbs', 
    assert_all_trsf_pole_nbs_found=True, 
    df_mp_serial_number_col='mfr_devc_ser_nbr', 
    df_mp_prem_nb_col='prem_nb', 
    df_mp_install_time_col='inst_ts', 
    df_mp_removal_time_col='rmvl_ts', 
    df_mp_trsf_pole_nb_col='trsf_pole_nb', 
    t_min_col='t_min', 
    t_max_col='t_max'
)
print(time.time()-start)

In [ ]:
active_SNs_df.equals(active_SNs_df_2)

In [ ]:
rcpo_df

In [ ]:
assert(sorted(rcpo_df.index.unique().tolist())==sorted(active_SNs_df.index.unique().tolist()))

In [ ]:
active_SNs_df.index.unique()

In [ ]:
rcpo_df_w_shit = rcpo_df.copy()

In [ ]:
rcpo_df_w_shit = add_xfmr_active_SNs_to_rcpo_df(
    rcpo_df=rcpo_df_w_shit, 
    trsf_pole_nbs_loc=trsf_pole_nbs_loc, 
    df_mp_curr=df_mp_curr, 
    df_mp_hist=df_mp_hist,
    time_infos_df=time_infos_df, 
    rcpo_df_to_time_infos_on = rcpo_df_to_time_infos_on, 
    time_infos_to_rcpo_df_on = time_infos_to_rcpo_df_on, 
    how=how, 
    rcpo_df_to_PNs_on = rcpo_df_to_PNs_on, 
    PNs_to_rcpo_df_on = PNs_to_rcpo_df_on
)

In [ ]:
rcpo_df_w_shit

## No Outage 2

In [ ]:
#OUTG
outg_rec_nb_col='trsf_pole_nb'
group_cols='trsf_pole_nb'
# group_cols=['outg_rec_nb', 'trsf_pole_nb']
addtnl_dropna_subset_cols=None

prem_nb_col='aep_premise_nb'
#-------------------------
paths_no_outg = Utilities.find_all_paths(
    base_dir=files_dir_no_outg, 
    glob_pattern=file_path_glob, 
    regex_pattern=file_path_regex
)
paths_no_outg = paths_no_outg[:10]
#-------------------------
end_events_df_no_outg = GenAn.read_df_from_csv_batch(
    paths=paths_no_outg, 
    cols_and_types_to_convert_dict=None, 
    to_numeric_errors='coerce', 
    make_all_columns_lowercase=True, 
    assert_all_cols_equal=True
)
#-------------------------
if (f'{outg_rec_nb_col}_gpd_for_sql' in end_events_df_no_outg.columns and 
    outg_rec_nb_col not in end_events_df_no_outg.columns):
    end_events_df_no_outg = end_events_df_no_outg.rename(columns={f'{outg_rec_nb_col}_gpd_for_sql':outg_rec_nb_col})
assert(outg_rec_nb_col in end_events_df_no_outg.columns)
#-------------------------
dropna_subset_cols = [outg_rec_nb_col]
if addtnl_dropna_subset_cols is not None:
    dropna_subset_cols.extend(addtnl_dropna_subset_cols)
end_events_df_no_outg = end_events_df_no_outg.dropna(subset=dropna_subset_cols)
#-------------------------
end_events_df_no_outg = AMIEndEvents.reduce_end_event_reasons_in_df(
    df=end_events_df_no_outg, 
    patterns_to_replace=patterns_to_replace
)
#-------------------------
rcpo_df_no_outg_2 = AMIEndEvents.get_reason_counts_per_group(
    end_events_df_no_outg, 
    group_cols=group_cols,
    group_freq=None, 
    serial_number_col='serialnumber', 
    reason_col='reason', 
    include_normalize_by_nSNs=False, 
    inclue_zero_counts=False,
    possible_reasons=None, 
    include_nSNs=True, 
    include_SNs=True, 
    prem_nb_col='aep_premise_nb', 
    include_nprem_nbs=False,
    include_prem_nbs=False, 
    return_form = dict(return_multiindex_outg_reason = False, 
                       return_normalized_separately  = False)    
)


In [ ]:
rcpo_df_no_outg_2.equals(rcpo_df_no_outg)

In [ ]:
rcpo_df_no_outg_2.shape

In [ ]:
rcpo_df = rcpo_df_no_outg_2.copy()

In [ ]:
rcpo_df.shape

In [ ]:
no_outg_time_infos_df_NEW = MECPOAn.get_bsln_time_interval_infos_df_from_summary_files(
    summary_paths=[AMIEndEvents.find_summary_file_from_csv(x) for x in paths_no_outg], 
    output_prem_nbs_col='prem_nbs', 
    output_t_min_col='t_min', 
    output_t_max_col='t_max', 
    make_addtnl_groupby_idx=True, 
    include_summary_paths=True
)

In [ ]:
no_outg_time_infos_df_NEW

In [ ]:
time_infos_df = no_outg_time_infos_df_NEW

In [ ]:
trsf_pole_nbs_idx_lvl=0
mp_df_curr_hist_no_outg_2 = MeterPremise.build_mp_df_curr_hist_for_xfmrs(trsf_pole_nbs=rcpo_df.index.get_level_values(trsf_pole_nbs_idx_lvl).tolist())
print(mp_df_curr_hist_no_outg_2['mp_df_curr'].shape)
print(mp_df_curr_hist_no_outg_2['mp_df_hist'].shape)

In [ ]:
df_mp_curr=mp_df_curr_hist_no_outg_2['mp_df_curr']
df_mp_hist=mp_df_curr_hist_no_outg_2['mp_df_hist']

In [ ]:
trsf_pole_nbs_loc = ('index', 'trsf_pole_nb')


# rcpo_df_to_time_infos_on = ['index_0']
rcpo_df_to_time_infos_on = [('index', 'trsf_pole_nb')]
time_infos_to_rcpo_df_on = ['index']
how='left'


rcpo_df_to_PNs_on = [('index', 'trsf_pole_nb')]
PNs_to_rcpo_df_on = ['index']
how='left'

In [ ]:
rcpo_df.index.nlevels

In [ ]:
start=time.time()
active_SNs_1 = get_active_SNs_for_xfmrs_in_rcpo_df(
    rcpo_df=rcpo_df, 
    trsf_pole_nbs_loc=trsf_pole_nbs_loc, 
    df_mp_curr=df_mp_curr, 
    df_mp_hist=df_mp_hist,
    time_infos_df=time_infos_df, 
    rcpo_df_to_time_infos_on = rcpo_df_to_time_infos_on, 
    time_infos_to_rcpo_df_on = time_infos_to_rcpo_df_on, 
    how='left', 
    rcpo_df_to_PNs_on = rcpo_df_to_PNs_on, 
    PNs_to_rcpo_df_on = PNs_to_rcpo_df_on, 
    addtnl_mp_df_curr_cols=None, 
    addtnl_mp_df_hist_cols=None, 
    return_SNs_col='SNs', 
    return_prem_nbs_col='prem_nbs', 
    assert_all_trsf_pole_nbs_found=True, 
    df_mp_serial_number_col='mfr_devc_ser_nbr', 
    df_mp_prem_nb_col='prem_nb', 
    df_mp_install_time_col='inst_ts', 
    df_mp_removal_time_col='rmvl_ts', 
    df_mp_trsf_pole_nb_col='trsf_pole_nb', 
    t_min_col='t_min', 
    t_max_col='t_max'
)
print(time.time()-start)

In [ ]:
active_SNs_1

In [ ]:
start=time.time()
active_SNs_2 = get_active_SNs_for_xfmrs_in_rcpo_df_v2(
    rcpo_df=rcpo_df, 
    trsf_pole_nbs_loc=trsf_pole_nbs_loc, 
    df_mp_curr=df_mp_curr, 
    df_mp_hist=df_mp_hist,
    time_infos_df=time_infos_df, 
    rcpo_df_to_time_infos_on = rcpo_df_to_time_infos_on, 
    time_infos_to_rcpo_df_on = time_infos_to_rcpo_df_on, 
    how='left', 
    rcpo_df_to_PNs_on = rcpo_df_to_PNs_on, 
    PNs_to_rcpo_df_on = PNs_to_rcpo_df_on, 
    addtnl_mp_df_curr_cols=None, 
    addtnl_mp_df_hist_cols=None, 
    return_SNs_col='SNs', 
    return_prem_nbs_col='prem_nbs', 
    assert_all_trsf_pole_nbs_found=True, 
    df_mp_serial_number_col='mfr_devc_ser_nbr', 
    df_mp_prem_nb_col='prem_nb', 
    df_mp_install_time_col='inst_ts', 
    df_mp_removal_time_col='rmvl_ts', 
    df_mp_trsf_pole_nb_col='trsf_pole_nb', 
    t_min_col='t_min', 
    t_max_col='t_max'
)
print(time.time()-start)

In [ ]:
active_SNs_2

In [ ]:
active_SNs_2.equals(active_SNs_1)

In [ ]:
rcpo_df_no_outg2_w_shit = rcpo_df.copy()

In [ ]:
# rcpo_df_no_outg2_w_shit = add_xfmr_active_SNs_to_rcpo_df(
#     rcpo_df=rcpo_df_no_outg2_w_shit, 
#     trsf_pole_nbs_loc=trsf_pole_nbs_loc, 
#     df_mp_curr=df_mp_curr, 
#     df_mp_hist=df_mp_hist,
#     time_infos_df=time_infos_df, 
#     rcpo_df_to_time_infos_on = rcpo_df_to_time_infos_on, 
#     time_infos_to_rcpo_df_on = time_infos_to_rcpo_df_on, 
#     how='left', 
#     rcpo_df_to_PNs_on = rcpo_df_to_PNs_on, 
#     PNs_to_rcpo_df_on = PNs_to_rcpo_df_on
# )


rcpo_df_no_outg2_w_shit = build_rcpo_df_norm_by_xfmr_active_nSNs(
    rcpo_df_raw=rcpo_df_no_outg2_w_shit, 
    trsf_pole_nbs_loc=trsf_pole_nbs_loc, 
    df_mp_curr=df_mp_curr, 
    df_mp_hist=df_mp_hist,
    time_infos_df=time_infos_df, 
    rcpo_df_to_time_infos_on = rcpo_df_to_time_infos_on, 
    time_infos_to_rcpo_df_on = time_infos_to_rcpo_df_on, 
    how='left', 
    rcpo_df_to_PNs_on = rcpo_df_to_PNs_on, 
    PNs_to_rcpo_df_on = PNs_to_rcpo_df_on
)

In [ ]:
rcpo_df_no_outg2_w_shit

In [ ]:
rcpo_df_no_outg

In [ ]:
rcpo_df_no_outg2_w_shit.equals(rcpo_df_no_outg)

In [ ]:
# THE V1 Results
print(active_SNs_2.equals(active_SNs_for_xfmrs_no_outg))
# print(active_SNs_2['prem_nbs'].apply(lambda x: sorted(x)).equals(xfmr_for_PNs_df_no_outg['prem_nbs'].apply(lambda x: sorted(x))))